# Biblioteca Financeira INCO

> – Dr. Bob Niedorf: How old is a person born in 1928?<br/>
– George Malley: Man or a woman?<br/>
– Dr. Bob Niedorf: Why?<br/>
– George Malley: Specifics, Bob.<br/>
– Dr. Bob Niedorf: Okay, one more time. How old is a MAN born in 1928?<br/>
– George Malley: Still alive?<br/>
– Dr. Bob Niedorf:  If a man is born in 1928, and he’s still alive, how old is he?<br/>
– George Malley: What month?<br/>
– Dr. Bob Niedorf: If a man was born October 3rd, 1928, and he’s still alive, how old is he?<br/>
– George Malley: What time?<br/>
– Dr. Bob Niedorf: 10 o’clock… PM!<br/>
– George Malley: Where?<br/>
– Dr. Bob Niedorf: Anywhere!<br/>
– George Malley: Well, let’s get specific, Bob! I mean, if the guy’s still alive, born in California, October 3rd, 1928, 10 PM, he’s 67 years, 9 months, 22 days, 14 hours, and…[takes Bob’s hand to see his wristwatch]… and 12 minutes. If he was born in New York, he’s 3 hours older, now isn’t he?<br/>
>
> — John Travolta, [Phenomenom](https://www.imdb.com/title/tt0117333).

Este caderno documenta a Biblioteca Financeira da INCO. Seu objetivo é definir com a maior especificidade possível o funcionamento de suas rotinas. Elas vão suportar uma combinação de operações – Bullet, Price, Juros mensais, e Livre – com indexadores – Pré-fixado, CDI, Poupança e IPCA. Mas com a ressalva de que Price não comporta indexadores flutuantes (CDI, Poupança e IPCA) dado que seus pagamentos devem ser fixos.

Observe que Livre é terminologia interna INCO, não é denominação usual de mercado. Foi criada para permitir operações de crédito com um nível máximo de flexibilidade.

Observe também que historicamente tivemos uma modalidade de operação que foi posteriormente abandonada, a Pool. Nossa base de código contém vestígios dela, especialmente nas tradutoras. Mas não são ativamente mantidas.

Além disso, os pagamentos gerados pelo motor financeiro deste caderno podem apresentar prestações de antecipação, inseridas na montagem do esqueleto do cronograma de acordo com a data em que ocorreu o evento de antecipação.

##  Conceitos

1. Amortização  
> Amortização é um processo de extinção de uma dívida através de pagamentos periódicos, que são realizados em função de um planejamento, de modo que cada prestação corresponde à soma do reembolso do capital ou do pagamento dos juros do saldo devedor, podendo ser o reembolso de ambos, sendo que os juros são sempre calculados sobre o saldo devedor.
>
> — [Amortização](https://pt.wikipedia.org/wiki/Amortização), Wikipédia.

2. Antecipação
> A antecipação é caracterizada pela amortização do principal fora do cronograma ordinário da modalidade de crédito e pode assumir caráter parcial ou total. A amortização de uma parte da dívida restante é chamada de parcial, enquanto a amortização que engloba toda essa dívida é chamada de total.

In [1]:
# Installing non-native libraries.
!python3 -m pip install typing-extensions python-dateutil requests holidays typeguard
!python3 -m pip install ipywidgets==7.7.2  # FIXME: Higher versions are not working.
!python3 --version

import typeguard

# Monkeypatch "typeguard". See "http://github.com/agronholm/typeguard/issues/364".
typeguard.typechecked = lambda x: x

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.10
    Uninstalling jupyterlab_widgets-3.0.10:
      Successfully uninstalled jupyterlab_widgets-3.0.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
Python 3.10.12


## Bullet

Também conhecido como Sistema de Amortização Americano Bullet – SAA Bullet.

Nesta modalidade o principal é devolvido em uma única parcela no final do prazo de  financiamento, e os juros de cada período são incorporados ao capital   (refinanciado) e também liquidados de uma única vez juntamente com o principal. Observe que o Sistema Americano pode ser reduzido a um Fluxo de Caixa com uma entrada e apenas uma amortização, com prazo de carência igual ao prazo do financiamento.

### Indexador Pré-fixado

Na Inco usa-se base diária considerando-se ano de 360 ou 365 dias. No entanto, a base de 365 foi abandonada pela Inco e só é suportada nesse motor para atender as operações antigas.

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{B} \\
J &= (FJ - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= P + J
\end{align}

Em que

* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $B$ é a base de cálculo diária, ou o número de dias corridos em 1 (um) ano; e
* $J$ é o valor do juros; e
* $P$ é o valor do principal; e
* $PMT$ é o valor da prestação.

### Indexador CDI

Na Inco usa-se base diária considerando-se ano de 252 dias.

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{252} \\
FDI &= \prod_{k=1}^n (1 + (PI \times \frac{TDI_k}{100})) \\
J &= (FJ \times FDI - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= P + J
\end{align}

Em que

* $FJ$ é o fator de juros (ou Spread); e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias úteis entre a data do início do rendimento e a data de cálculo; e
* $FDI$ é o fator de DI; e
* $PI$ é o percentual do índice a ser considerado; e
* $TDI$ é a taxa diária percentual do CDI, divulgada pelo BACEN; e
* $n$ é o número de taxas DI divulgadas no período de rendimento; e
* $k$ é o número de ordem da taxa DI; e
* $J$ é o valor do juros; e
* $P$ é o valor do principal; e
* $PMT$ é o valor da prestação.

### Indexador Poupança

Na Inco usa-se base diária considerando-se ano de 360 dias.

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{360} \\
FP &= \prod_{k=1}^n (1 + (PI \times  \frac{TP_k}{100})) \\
J &= (FJ \times FP - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= P + J
\end{align}

Em que

* $FJ$ é o fator de juros (ou Spread); e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $FP$ é o fator de Poupança; e
* $PI$ é o percentual do índice a ser considerado; e
* $TP$ é a taxa mensal percentual da Poupança, divulgada pelo BACEN; e
* $n$ é o número de taxas divulgadas no período de rendimento, sendo consideradas só aquelas cujos dias de início e término sejam iguais ao dia do aniversário (ou liquidação) da operação; e
* $k$ é o número de ordem da taxa da Poupança; e
* $J$ é o valor do juros; e
* $P$ é o valor do principal; e
* $PMT$ é o valor da prestação.

### Indexador ICPA

Na Inco usa-se base diária considerando-se ano de 360 dias.

Cálculo da correção.

\begin{align}
FC &= \prod_{k=1}^n (1 + \frac{VM_k}{100})^\frac{dp_k}{dt_k} \\
C &= (FC - 1) \times P
\end{align}

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{360} \\
J &= (FJ - 1) \times (P + C)
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= P + C + J
\end{align}

Em que

* $FC$ é o fator de correção do IPCA; e
* $VM$ é a variação mensal percentual do IPCA, divulgada pelo BACEN; e
* $dp_k$ é o número de dias corridos entre a data de início da correção e a data do cálculo, ou a data de fim da correção, caso seja menor; e
* $dt_k$ é o número de dias corridos entre a data de início da correção e a data de fim da correção; e
* $n$ é o número de taxas de correção divulgadas no período de rendimento; e
* $k$ é o número de ordem da taxa de correção; e
* $C$ é o valor da correção; e
* $P$ é o valor do principal; e
* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $J$ é o valor do juros; e
* $PMT$ é o valor da prestação.

## Juros mensais

Também conhecido como Sistema de Amortização Americano Padrão – SAA Pardão.

Nesta modalidade, o principal é também devolvido em uma única parcela no vencimento e os juros são pagos no final de cada período.

### Indexador Pré-fixado

Na Inco usa-se base diária considerando-se mês de 30 dias e ano de 360 dias.

Cálculo do juros.

\begin{align}
FJ &= [(1 + \frac{APY}{100})^\frac{30}{360}]^\frac{dp}{dt} \\
J &= (FJ - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &=
\left\{
\begin{array}{cl}
    J & i \ne m \\
    P + J & i = m.
\end{array}
\right.
\end{align}

Em que

* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $dt$ é o número de dias corridos entre a data do início do rendimento e a data do fim do rendimento; e
* $J$ é o valor do juros; e
* $P$ é o valor do principal; e
* $PMT$ é o valor da prestação; e
* $i$ é o número da prestação; e
* $m$ é o prazo do rendimento, em meses.

### Indexador CDI

Na Inco usa-se base diária considerando-se ano de 252 dias.

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{252} \\
FDI &= \prod_{k=1}^n (1 + (PI \times \frac{TDI_k}{100})) \\
J &= (FJ \times FDI - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &=
\left\{
\begin{array}{cl}
    J & i \ne m \\
    P + J & i = m.
\end{array}
\right.
\end{align}

Em que

* $FJ$ é o fator de juros (ou Spread); e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias úteis entre a data do início do rendimento e a data de cálculo; e
* $FDI$ é o fator de DI; e
* $PI$ é o percentual do índice a ser considerado; e
* $TDI$ é a taxa diária percentual do CDI, divulgada pelo BACEN; e
* $n$ é o número de taxas DI divulgadas no período de rendimento; e
* $k$ é o número de ordem da taxa DI; e
* $J$ é o valor do juros; e
* $P$ é o valor do principal; e
* $PMT$ é o valor da prestação; e
* $i$ é o número da prestação; e
* $m$ é o prazo do rendimento, em meses.

### Indexador ICPA

Na Inco usa-se base diária considerando-se mês de 30 dias e ano de 360 dias.

Cálculo da correção.

\begin{align}
FC &= \prod_{k=1}^n (1 + \frac{VM_k}{100})^\frac{dp_k}{dt_k} \\
C &= (FC - 1) \times P
\end{align}

Cálculo do juros.

\begin{align}
FJ &= [(1 + \frac{APY}{100})^\frac{30}{360}]^\frac{dp}{dt} \\
J &= (FS - 1) \times (P + C)
\end{align}

Cálculo da prestação.

\begin{align}
PMT &=
\left\{
\begin{array}{cl}
    J & i \ne m \\
    P + C + J & i = m.
\end{array}
\right.
\end{align}

Em que

* $FC$ é o fator de correção do IPCA; e
* $VM$ é a variação mensal percentual do IPCA, divulgada pelo BACEN; e
* $C$ é o valor da correção; e
* $dp_k$ é o número de dias corridos entre a data de início da correção e a data do cálculo, ou a data de fim da correção, caso seja menor; e
* $dt_k$ é o número de dias corridos entre a data de início da correção e a data de fim da correção; e
* $n$ é o número de taxas de correção divulgadas entre um (1) mês anterior à data de início da primeira prestação e um (1) mês anterior à data de início da prestação calculada; e
* $k$ é o número de ordem da taxa de correção; e
* $P$ é o valor do principal; e
* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $dt$ é o número de dias corridos entre a data do início do rendimento e a data do fim do rendimento; e
* $J$ é o valor do juros; e
* $PMT$ é o valor da prestação; e
* $i$ é o número da prestação; e
* $m$ é o prazo do rendimento, em meses.


## Price

Tabela Price, TP, ou Sistema Francês de Amortização – SFA.

> “A denominação do Sistema de Amortização Francês vem do fato de ter sido utilizado primeiramente na França, no século XIX.”
>
> — SAMANEZ, Carlos Patricio. Matemática Financeira: Aplicações à análise de investimentos. São Paulo, Pearson Education do Brasil, ed 3, 2002.

Na Tabela PRICE, todas as prestações são iguais, periódicas e sucessivas.

### Indexador Pré-fixado

Na Inco usa-se base diária considerando-se mês de 30 dias e ano de 360 dias.

Cálculo da prestação.

\begin{align}
TM &= (1 + \frac{APY}{100})^\frac{30}{360} - 1 \\
PMT &= \frac{P \times TM}{1 - \frac{1}{(1 + TM)^n}}
\end{align}

Cálculo do juros.

\begin{align}
FJ &= [(1 + \frac{APY}{100})^\frac{30}{360}]^\frac{dp}{dt} \\
J &= (FJ - 1) \times BAL
\end{align}

Cálculo da amortização.

\begin{align}
A &= PMT - J_f
\end{align}

Em que

* $TM$ é a taxa mensal de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $PMT$ é o valor da prestação; e
* $P$ é o valor do principal; e
* $n$ é o prazo do rendimento, em meses.
* $FJ$ é o fator de juros; e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $dt$ é o número de dias corridos entre a data do início do rendimento e a data do fim do rendimento; e
* $J$ é o valor do juros; e
* $BAL$ é o valor do saldo devedor (Balance).
* $A$ é o valor da amortização do principal; e
* $J_f$ é o valor final do juros, calculado na data de fim do rendimento.

## Livre

Nesta modalidade o pagamento do principal e do juros é customizável. Dessa forma, é possível determinar o percentual de amortização e se haverá, ou não, pagamento do juros acumulado em cada prestação.

### Indexador Pré-fixado

Na Inco usa-se base diária considerando-se mês de 30 dias e ano de 360 dias.

Cálculo da amortização.

\begin{align}
A &= P \times \frac{PCT}{100}
\end{align}

Cálculo de juros.

\begin{align}
FJ &= [(1 + \frac{APY}{100})^\frac{30}{360}]^\frac{dp}{dt} \\
J &= (FJ - 1) \times BAL
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= A + J
\end{align}

Em que

* $A$ é o valor da amortização do principal; e
* $P$ é o valor do principal; e
* $PCT$ é o percentual de amortização; e
* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $dt$ é o número de dias corridos entre a data do início do rendimento e a data do fim do rendimento; e
* $J$ é o valor do juros; e
* $BAL$ é o valor do saldo devedor (Balance) anterior à amortização; e
* $PMT$ é o valor da prestação.

### Indexador CDI

Na Inco usa-se base diária considerando-se ano de 252 dias.

Cálculo da amortização.

\begin{align}
A &= P \times \frac{PCT}{100}
\end{align}

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{252} \\
FDI &= \prod_{k=1}^n (1 + (PI \times \frac{TDI_k}{100})) \\
J &= (FJ \times FDI - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= A + J
\end{align}

Em que

* $A$ é o valor da amortização do principal; e
* $P$ é o valor do principal; e
* $PCT$ é o percentual de amortização; e
* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias úteis entre a data do início do rendimento e a data de cálculo; e
* $FDI$ é o fator de DI; e
* $PI$ é o percentual do índice a ser considerado; e
* $TDI$ é a taxa diária percentual do CDI, divulgada pelo BACEN; e
* $n$ é o número de taxas DI divulgadas no período de rendimento; e
* $k$ é o número de ordem da taxa DI; e
* $J$ é o valor do juros; e
* $B$ é o valor do saldo devedor (Balance) anterior à amortização; e
* $PMT$ é o valor da prestação.

### Indexador IPCA

Na Inco usa-se base diária considerando-se mês de 30 dias e ano de 360 dias.

Cálculo da amortização.

\begin{align}
A &= P \times \frac{PCT}{100}
\end{align}

Cálculo da correção.

\begin{align}
FC &= \prod_{k=1}^n (1 + \frac{VM_k}{100})^\frac{dp_k}{dt_k} \\
C &= (FC - 1) \times A
\end{align}

Cálculo do juros.

\begin{align}
FJ &= [(1 + \frac{APY}{100})^\frac{30}{360}]^\frac{dp}{dt} \\
J &= (FS - 1) \times (BAL + C)
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= A + C + J
\end{align}
Em que

* $A$ é o valor da amortização do principal; e
* $P$ é o valor do principal; e
* $PCT$ é o percentual de amortização; e
* $FC$ é o fator de correção do IPCA; e
* $VM$ é a variação mensal percentual do IPCA, divulgada pelo BACEN; e
* $C$ é o valor da correção; e
* $BAL$ é o valor do saldo devedor (Balance) anterior à amortização; e
* $dp_k$ é o número de dias corridos entre a data de início da correção e a data do cálculo, ou a data de fim da correção, caso seja menor; e
* $dt_k$ é o número de dias corridos entre a data de início da correção e a data de fim da correção; e
* $n$ é o número de taxas de correção divulgadas entre um (1) mês anterior à data de início da primeira prestação e um (1) mês anterior à data de início da prestação calculada; e
* $k$ é o número de ordem da taxa de correção; e
* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $dt$ é o número de dias corridos entre a data do início do rendimento e a data do fim do rendimento; e
* $J$ é o valor do juros; e
* $PMT$ é o valor da prestação.

## Observações

### Sobre o fator de correção para o IPCA

As taxas de variação mensal ($VM$) utilizadas para calcular o fator de correção ($FC$) são determinadas por uma data base ($D$), o período ($p$) a ser considerado, em meses, e um deslocamento ($s$), também mensal, de modo que:

\begin{align}
s &=
\left\{
\begin{array}{cl}
    0 & AUTO \\
    1 \: mês & M-1 \\
    2 \: meses & M-2.
\end{array}
\right.
\end{align}

\begin{align}
D - s \le d_{VM} \le D + p - s
\end{align}

Onde $d_{VM}$ é a data da taxa de variação analisada.

### Sobre a antecipação parcial para modalidades com amortizações periódicas

A inserção de uma antecipação parcial na montagem do cronograma para alguma das modalidades de crédito que possuem amortizações periódicas no fluxo de pagamentos ordinário (Price ou Livre) faz com que os percentuais de amortização do principal posteriores à essa antecipação precisem ser atualizados. Essa atualização é feita de forma que o novo percentual de amortização ($P_{novo}$), de uma prestação arbitrária, deve ser igual ao percentual antigo ($P_{antigo}$) multiplicado por um fator de ajuste ($F$):

\begin{align}
F &= \frac{A_{corrente}}{A_{ordinária}} \\
P_{novo} &= P_{antigo} * F
\end{align}

Onde $A_{corrente}$ é o percentual de amortização restante do fluxo de pagamentos corrente, em cima do principal e incluindo as amortizações extraordinárias (antecipações), e $A_{ordinário}$ é o percentual de amortização restante do fluxo de pagamentos ordinário.

# Algoritmo

In [2]:
# Copyright (C) Inco - All Rights Reserved.
#
# Written by Rafael Viotti <viotti@inco.vc>,
#            Gustavo Zille <gustavo.zille@inco.vc>, April 2022.
#
# Unauthorized copying of this file, via any medium, is strictly prohibited.
# Proprietary and confidential.
#
# [FINCORE V3]
#
# Penso que esse módulo está em sua terceira geração. Houve também uma geração zero, que antecedeu a criação desse
# módulo, quando as rotinas de cálculos financeiros eram auxiliares do módulo "util". Naquela época contemplávamos
# apenas operações pré-fixadas, fluxos normais.
#
# A primeira geração do Fincore nasceu em 22 de Março de 2022, no repositório de suporte, na forma de um caderno
# Jupyter, "lab-000.ipynb". Ver a revisão "6f12bab6dc". Em Abril foi integrado à retaguarda da INCO com o nome
# "util.cli.fin_lib".
#
# A segunda geração veio de uma refatoração em Novembro de 2022, revisão "ca58a71c19". Nessa época a biblioteca sofria
# de anemia. Os tipos de dados não tinham nomes, nem atributos. Toda entrada e saída acontecia por meio de tuplas, ou
# "collections.namedtuple". A API pública era extremamente confusa, com factories mal projetadas.
#
# Em Julho de 2023, fiz nova refatoração, agora com foco de tornar universais os algoritmos do Fincore, que entra em
# sua terceira geração. Revisão "42bdf101c5" e subsequentes. Ainda tínhamos limitações arbitrárias de funcionamento, e
# erros. O que era mera inconveniência, tornou-se impedimento para novas operações da INCO. Segue um resumo do que
# mudou.
#
#   • O núcleo ficou melhor estruturado e documentado. Opera em três fases bem definidas.
#
#     1. Calcula fatores de "spread" e correção.
#     2. Registra valores da iteração corrente do cronograma: principal e percentual de amortização, juros, correção etc.
#     3. Cria a instância de saída da iteração, Payment / PriceAdjustedPayment, e arredondamentos.
#
#   • Ficou mais genérico.
#
#     • Contempla 100% dos casos na modalidade Livre.
#       • Exemplo: pagamentos trimestrais.
#
#     • Contempla mais fluxos alternativos.
#       • Exemplo: pagamento em carência.
#
#   • Erros foram corrigidos.
#
#     • Não incorpora juros em principal.
#       • Isso corrige operações com carência.
#
#     • Não projeta correção monetária após data de cálculo.
#       • Isso corrige o vencimentário 3040.
#
#   • API Pública melhorou. A antecipação ficou mais semântica.
#
#     • Leva o valor antecipado, não um valor percentual.
#       • Morte da rotina "fincore.get_remaining_amortization".
#
#   • O módulo diminuiu quase 15% em tamanho. De ~1600 para ~1400 linhas de código.
#
# [ANNOTATED_TYPES]
#
# Use annotated types instead of explicit checking in code – http://stackoverflow.com/a/72563242.
#
# [WEAKNESSES]
#
#   • O Fincore trabalha com datas, não data/hora. Por esse motivo, se for preciso amortizar duas vezes em um mesmo dia,
#     sua ordem fica implícita na tabela de amortizações. Idealmente, esse módulo seria agnóstico à mensurações de tempo
#     decorrido. O campo "date" da classe Amortization deveria aceitar tanto uma data, como uma data/hora. A saída
#     do "get_payments_table" deveria se ajustar de acordo com o tipo de dado usado na entrada. Internamente, o Fincore
#     poderia simplesmente descartar a informação de hora, e lidar apenas com datas.
#
#   • Arredondamento não está documentado. Explicitar que o Fincore faz arredondamentos na saída, ou seja, instâncias de
#     Payment ou PriceAdjustedPayment sempre estão arredondados. Mas não há arredondamentos internos. Exemplificar.
#
#   • O comportamento da biblioteca em caso de índice negativo poderia ser personalizável.
#
#   • Como essa biblioteca se comporta em caso de cálculos futuros? Hoje ela faz projeção automaticamente. Isso poderia
#     ser parametrizável. A rotina que calcula a tabela de pagamentos poderia falhar caso um índice não fosse encontrado
#     (data no futuro).
#
#   • Como fica o fator de correção menor que zero? Hoje, quando há antecipação, não há dedução de correção caso ela já
#     seja negativa. Ou seja, o valor antecipado vai ser deduzido dos juros e depois do principal. Isso também poderia
#     ser parametrizável.
#
#   • Por arredondar valores apenas na saída, não se deve reusar valores de saída de um cronograma de pagamento como
#     entrada. Por exemplo, não se deve rodar a rotina nuclear, "get_payments_table", com uma data de cálculo e somar o
#     valor bruto com o saldo devedor de saída para se obter o valor de uma suposta antecipação total
#
#       _tail = lambda iterable: iter(collections.deque(iterable, maxlen=1))
#       kargs = { … }  # Opções de um cronograma de amortização Livre, incluindo uma data de cálculo.
#
#       # Executa o Fincore (1ª vez) para obter a posição na data de cálculo.
#       row = next(_tail(fincore.build(**kargs)))
#
#       # Aqui, "row.raw + row.bal" seria o valor de uma antecipação total na data de cálculo, mas nem sempre vai ser.
#       kargs['insertions'].append(fincore.Amortization.Bare(date=datetime.date(…), value=row.raw + row.bal))
#
#       # Executa o Fincore (2ª vez). Teoricamente, o cronograma de pagamento terminaria na antecipação.
#       row = next(_tail(fincore.build(**kargs)))
#
#     No exemplo anterior, tanto "row.raw" quanto "row.bal" podem ter sido arredondados para cima, e o valor da
#     antecipação seria superior ao saldo remanescente do empréstimo em um centavo. Melhor é ser explícito, e usar
#     "fincore.Amortization.Bare.MAX_VALUE" para sinalizar que todo o valor em aberto do empréstimo deve ser
#     antecipado.
#
#       fincore.Amortization.Bare(date=datetime.date(…), value=fincore.Amortization.Bare.MAX_VALUE)
#
#     De forma análoga, se ocorrer arrendondamento para baixo, o valor da antecipação não quitará toda operação. Um
#     valor inadequado para uma antecipação total poderia passar despercebido.
#
#     FIXME: se o Fincore fizesse arredondamentos internamente, o problema descrito acima não aconteceria. Seria
#     possível realimentar um cronograma de amortizações com valores provenientes de um cronograma de pagamentos gerado
#     por uma invocação prévia da rotina nuclear.
#
# [TODO]
#
#   • Implementar validações para prevenir entradas do tipo "Amortization.Bare" não quantizadas nas rotinas auxiliares,
#     as "factories", e em "get_payments_table".
#
#   • Implementar uma memória para o IPCA pago e para o acumulado, em caso de antecipação, de forma semelhante aos juros
#     pagos. Lembrar que os juros se compõem, mas o IPCA não.
#

'''
Núcleo financeiro INCO, Fincore.

Você pode encontrar o código desse módulo no caderno IPython "lab-000.ipynb" que encontra-se no repositório de suporte.
Mas fique atento, pois o caderno pode estar desatualizado.
'''

# Python.
import sys
import math
import types
import typing as t
import decimal
import logging
import calendar
import datetime
import functools
import itertools
import contextlib
import dataclasses

# Libs.
import typeguard
import dateutil.relativedelta

# Logger object.
_LOG = logging.getLogger('fincore')

# Zero as decimal.
_0 = decimal.Decimal()

# One as decimal.
_1 = decimal.Decimal(1)

# Centi factor.
_CENTI = decimal.Decimal('0.01')

# Centesimal quantization.
_Q = functools.partial(decimal.Decimal.quantize, exp=_CENTI, rounding=decimal.ROUND_HALF_UP)

# Generic type.
_T = t.TypeVar('_T')

# A month.
_MONTH = dateutil.relativedelta.relativedelta(months=1)

# Mathematical tolerance for numerical operations.
_MATH_TOL = 1e-9

# Tabela de IR para aplicações de renda fixa.
#
# Sei que "sys.maxsize" não é o valor máximo de um numero inteiro no Python 3,
# mas sim o tamanho máximo da palavra na arquitetura em que o interpretador
# estiver executando.
#
#   https://stackoverflow.com/a/7604981
#
# Não me importo. O que quero na última entrada da tabela é apenas um número
# suficientemente grande.
#
_REVENUE_TAX_BRACKETS = [
    (0, 180, decimal.Decimal('0.225')),
    (180, 360, decimal.Decimal('0.2')),
    (360, 720, decimal.Decimal('0.175')),
    (720, sys.maxsize, decimal.Decimal('0.15'))
]

# Supported operation modes.
_OP_MODES = t.Literal['Bullet', 'Juros mensais', 'Price', 'Livre']

# Variable rate indexes.
_VR_INDEX = t.Literal['CDI', 'Poupança']

# Price level indexes.
_PL_INDEX = t.Literal['IPCA', 'IGPM']

# Price level indexes, range shifters: AUTO, M-1, M-2.
_PL_SHIFT = t.Literal['AUTO', 'M-1', 'M-2']

# Capitalisation methods, daily and monthly. Defines how DP/DT calculations are performed.
_CAPITALISATION = t.Literal['252', '360', '365', '30/360']

# Daily capitalisation methods.
_DAILY_CAPITALISATION = t.Literal['252', '360', '365']

# Gain output mode.
_GAIN_OUTPUT_MODE = t.Literal['current', 'deferred', 'settled']

# Helpers. {{{
@typeguard.typechecked
def _delta_months(d1: datetime.date, d2: datetime.date) -> int:
    '''
    Returns the number of months between two given dates, D1 and D2.

    This function will subtract D2 from D1, i.e., it will roughly perform
    "(D1 - D2).months". The day of the month is completely ignored in the
    calculation.

    >>> from datetime import date
    >>>
    >>> _delta_months(date(2018, 5, 5), date(2018, 5, 15))
    0
    >>> _delta_months(date(2019, 11, 1), date(2019, 10, 10))
    1
    >>> _delta_months(date(2023, 5, 3), date(2022, 5, 4))
    12
    >>> _delta_months(date(2000, 6, 1), date(1996, 6, 19))
    48

    Deltas will be negative if D2 > D1.

    >>> _delta_months(date(2018, 5, 5), date(2018, 11, 5))
    -6
    '''

    return (d1.year - d2.year) * 12 + d1.month - d2.month

@typeguard.typechecked
def _date_range(start_date: datetime.date, end_date: datetime.date) -> t.Iterable[datetime.date]:
    iterator = start_date

    while iterator < end_date:
        yield iterator

        iterator += datetime.timedelta(days=1)

# See also https://stackoverflow.com/a/27667421 and https://stackoverflow.com/a/14994380.
@typeguard.typechecked
def _diff_days_to_same_day_on_prev_month(base: datetime.date) -> int:
    '''
    Returns the amount of days between a given date and the same day on the previous month.

    >>> lst = []
    >>> for month in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1]:
    ...     lst.append(_diff_days_to_same_day_on_prev_month(datetime.date(2022, month, 10)))
    >>> lst
    [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    In essence, what this function does is return the amount of days the previous month has. Hence, it has four
    possible outcomes: 28, 29, 30, and 31 days.
    '''

    return (base - (base - _MONTH)).days

@typeguard.typechecked
def _diff_surrounding_dates(base: datetime.date, day_of_month: int) -> int:
    '''
    Returns the amount of days between two dates derived from a base date.

    Given a base date, this function will find two dates that surround it on
    the specified day of the month. For example, if the base date is
    2022-06-12, and the day of the month is 15, then de surrounding dates are
    2022-05-15 and 2022-06-15. It will then return the difference between these
    dates, in days.

    >>> from datetime import date
    >>>
    >>> _diff_surrounding_dates(date(2022, 6, 12), 24)
    31

    So, this function has four possible outputs: 28, 29, 30, and 31 days.

    >>> _diff_surrounding_dates(date(1999, 2, 25), 10)
    28
    >>> _diff_surrounding_dates(date(2000, 3, 20), 25)
    29
    >>> _diff_surrounding_dates(date(1900, 11, 20), 19)
    30
    >>> _diff_surrounding_dates(date(2050, 7, 2), 1)
    31
    >>> _diff_surrounding_dates(date(2050, 7, 2), 2)
    31
    >>> _diff_surrounding_dates(date(2050, 7, 2), 3)
    30

    It should be possible to find a date before the base date, otherwise an
    error is thrown.

    >>> _diff_surrounding_dates(date.min, 3)
    Traceback (most recent call last):
        ...
    ValueError: can't find a date prior to the base of 0001-01-01 on day 3
    '''

    if base.day >= day_of_month or base >= datetime.date(1, 2, 1):
        d01 = base.replace(day=day_of_month)
        d02 = d01 + _MONTH if base.day >= day_of_month else d01 - _MONTH
        dff = d02 - d01 if base.day >= day_of_month else d01 - d02

        return dff.days

    else:
        raise ValueError(f"can't find a date prior to the base of {base} on day {day_of_month}")

@typeguard.typechecked
def _interleave(a: t.Iterable[_T], b: t.Iterable[_T], *, key: t.Callable[..., t.Any] = lambda x: x) -> t.Iterable[types.SimpleNamespace]:
    '''
    Interleave two ordered iterables into another, also ordered, iterable.

    Both source iterables, A and B, should produce items in order.

    >>> list(_interleave([], []))
    []

    >>> [(x.index_a, x.from_a, x.index_b, x.from_b, x.item) for x in _interleave([1, 3, 5], [2, 4, 6])]  # doctest: +NORMALIZE_WHITESPACE
    [(0, True,  0, False, 1),
     (1, False, 0, True,  2),
     (1, True,  1, False, 3),
     (2, False, 1, True,  4),
     (2, True,  2, False, 5),
     (2, False, 2, True,  6)]

    >>> list(_interleave([1, 5, 3], [2, 4, 6]))
    Traceback (most recent call last):
        ...
    ValueError: iterable A is not ordered

    When two items arising from A and B compare equally, the one from B have precedence over the one from A. This is
    because iterable A represents ordinary amortizations, whilst iterable B produces advance payments. In finance,
    advancements are usually processed first.

    >>> a = [1, 4, 6, 22, 55, 1232]
    >>> b = [4, 9, 11, 344, 999]
    >>> [(x.index_a, x.from_a, x.index_b, x.from_b, x.item) for x in _interleave(a, b)]  # doctest: +NORMALIZE_WHITESPACE
    [(0, True,  0, False, 1),
     (1, False, 0, True,  4),
     (1, True, 1, False,  4),
     (2, True,  1, False, 6),
     (3, False, 1, True,  9),
     (3, False, 2, True,  11),
     (3, True,  3, False, 22),
     (4, True,  3, False, 55),
     (5, False, 3, True,  344),
     (5, False, 4, True,  999),
     (5, True,  4, False, 1232)]

    But when a single source iterable produces items that compare equally, an exception is thrown. This generator was
    designed to interleave scheduled payments, A, with unscheduled payments, B. It is unusual to have more than one
    unscheduled payment on a given day.

    >>> a = [1, 5, 5, 6, 22, 90, 1000]
    >>> b = [5, 5, 9, 11, 82, 829]
    >>> list(_interleave(a, b))
    Traceback (most recent call last):
        ...
    ValueError: iterable B, item “5” found multiple times

      ADENDO: Aqui reside o nosso bloqueio contra duas antecipações no mesmo dia, dentre outros. Isso é bom, pois no
      ADENDO: momento as controladoras estão pouco testadas, e já houve tentativa de submeter uma única antecipação
      ADENDO: mais de uma vez. Aconteceu com a Mariana Castro no dia 01 de Agosto de 2023. Ela teve um fluxo de geração
      ADENDO: de antecipações totais interrompidos por um erro de requisição. Ao tentar resumir o processo, em uma
      ADENDO: requisição subsequente, a controladora passou novamente por investimentos que já possuíam a antecipação
      ADENDO: gerada, e tentou inseri-la novamente. Se não fosse essa proteção, teríamos um erro de esquema de dados.
      ADENDO:
      ADENDO: Assim que tivermos melhor cobertura nas controladoras de geração de antecipação, pagamento parcial,
      ADENDO: pagamento de prestações em lote etc, podemos remover essa limitação. Afinal de contas, ela é arbitrária.
      ADENDO: Do ponto de vista do Fincore, a ordem de duas antecipações em uma mesma data é dada pela posição na lista.
      ADENDO: Também é necessário ter casos de testes para antecipações consecutivas com os diversos indexadores
      ADENDO: suportados pelo Fincore, antes da remoção dessa trava.
      ADENDO:
      ADENDO: Vale lembrar que se há multiplas antecipações no mesmo dia, a solução simples seria somar os seus valores
      ADENDO: brutos.

    A key function can be used to establish comparison between elements.

    >>> a = [{'x': 500, 'date': '2022-02-05'}, {'x': 100, 'date': '2022-04-05'}, {'x': -50, 'date': '2022-06-05'}]
    >>> b = [{'y': 'a', 'date': '2021-02-05'}, {'y': 'b', 'date': '2021-04-05'}, {'y': 'c', 'date': '2021-06-05'}]
    >>> [(x.index_a, x.from_a, x.index_b, x.from_b, x.item) for x in _interleave(a, b, key=lambda x: x['date'])]  # doctest: +NORMALIZE_WHITESPACE
    [(0, False, 0, True,  {'y': 'a', 'date': '2021-02-05'}),
     (0, False, 1, True,  {'y': 'b', 'date': '2021-04-05'}),
     (0, False, 2, True,  {'y': 'c', 'date': '2021-06-05'}),
     (0, True,  2, False, {'x': 500, 'date': '2022-02-05'}),
     (1, True,  2, False, {'x': 100, 'date': '2022-04-05'}),
     (2, True,  2, False, {'x': -50, 'date': '2022-06-05'})]
    '''

    val_a = val_b = sav_a = sav_b = None
    idx_a = idx_b = -1

    iter_a = iter(a)
    iter_b = iter(b)

    with contextlib.suppress(StopIteration):
        val_a = next(iter_a)

        idx_a += 1

    with contextlib.suppress(StopIteration):
        val_b = next(iter_b)

        idx_b += 1

    while val_a or val_b:
        if sav_a and val_a and key(sav_a) > key(val_a):
            raise ValueError('iterable A is not ordered')

        elif sav_a and val_a and key(sav_a) == key(val_a):
            raise ValueError(f'iterable A, item “{sav_a}” found multiple times')

        elif sav_b and val_b and key(sav_b) > key(val_b):
            raise ValueError('iterable B is not ordered')

        elif sav_b and val_b and key(sav_b) == key(val_b):  # Ver o adendo na "docstring" da rotina. FIXME.
            raise ValueError(f'iterable B, item “{sav_b}” found multiple times')

        if val_a and val_b and key(val_a) == key(val_b):
            sav_b = val_b

            yield types.SimpleNamespace(index_a=idx_a, from_a=False, index_b=idx_b, from_b=True, item=val_b)

            with contextlib.suppress(StopIteration):
                val_b = None
                val_b = next(iter_b)

                idx_b += 1

        elif val_a and val_b and key(val_a) < key(val_b):
            sav_a = val_a

            yield types.SimpleNamespace(index_a=idx_a, from_a=True, index_b=idx_b, from_b=False, item=val_a)

            with contextlib.suppress(StopIteration):
                val_a = None
                val_a = next(iter_a)

                idx_a += 1

        elif val_a and val_b:
            sav_b = val_b

            yield types.SimpleNamespace(index_a=idx_a, from_a=False, index_b=idx_b, from_b=True, item=val_b)

            with contextlib.suppress(StopIteration):
                val_b = None
                val_b = next(iter_b)

                idx_b += 1

        elif val_a:
            sav_a = val_a

            yield types.SimpleNamespace(index_a=idx_a, from_a=True, index_b=idx_b, from_b=False, item=val_a)

            with contextlib.suppress(StopIteration):
                val_a = None
                val_a = next(iter_a)

                idx_a += 1

        else:
            sav_b = val_b

            yield types.SimpleNamespace(index_a=idx_a, from_a=False, index_b=idx_b, from_b=True, item=val_b)

            with contextlib.suppress(StopIteration):
                val_b = None
                val_b = next(iter_b)

                idx_b += 1
# }}}

# Public API. Main classes. {{{
@dataclasses.dataclass
class PriceLevelAdjustment:
    '''
    A price level adjustment specification for amortization schedules.

    This object has five fields.

      1. The market index used on this amortization. For now, only IPCA is supported. IGPM is planned.

      2. The first, or base date, of the date range used on the calculation of the price adjustment factor of the
      amortization. The factor will be the monthly accumulation of N indexes from the base date, where N is the
      range size, or period (see below).

      3. The period, or how many monthly indexes from the base date are accumulated to calculate the adjustment
      factor of the amortization. Should be greater than, or equal to one. Consider that the base date is March
      first 2022, and that the period is five.

             1          2          3          4          5
             ├──────────┼──────────┼──────────┼──────────┼──────────> 2022
            Mar        Apr        May        Jun        Jul

         In the example above, indexes from March to July would be accumulated.

      4. A shift, in months, for the price adjustment accumulation range.

        ▪︎ AUTO, don't shift.

        ▪︎ M-1 means start accumulating one month prior to the base date.

        ▪︎ M-2 means start accumulating two months prior to the base date.

         These are the effects of each shift option when applied to the previous example.

            -2         -1          1          2          3          4          5
             ├──────────├──────────┼──────────┼──────────┼──────────┼──────────┼──────────> 2022
                                  Mar        Apr        May        Jun        Jul    AUTO

                        Feb       Mar        Apr        May        Jun               M-1

            Jan         Feb       Mar        Apr        May                          M-2

      5. A boolean flag indicating whether the price adjustment will be amortized or incorporated in the debt.
    '''

    code: _PL_INDEX

    base_date: t.Optional[datetime.date] = None

    period: int = 0

    shift: _PL_SHIFT = 'M-1'

    amortizes_adjustment: bool = True

@dataclasses.dataclass
class DctOverride:
    '''
    An override for the DCT calculation. Has three fields.

      1. The date of the previous scheduled amortization.

      2. The date of the next scheduled amortization.

      3. A boolean stating if this override information predates the first scheduled amortization.
    '''

    date_from: datetime.date

    date_to: datetime.date

    predates_first_amortization: bool

@dataclasses.dataclass
class Amortization:
    '''
    A entry of an amortization schedule.

    Establishes what fraction of the initial debt should be paid at a given date.

    Some loans are more intricate than others: those with variable interest rates, price level adjustments, grace
    periods etc. The basic Amortization class does not provide enough information to create payment schedules for such
    cases. Enter the extension fields.

      • "price_level_adjustment", which is a “PriceLevelAdjustment” instance.

      • "dct_override", an override for the DCT calculation. See “DctOverride” below.

    The DCT override field is specific for 30/360 specs. We need to store extra override information for the DCT
    calculation to ensure that DCT is always the amount of days between two scheduled payments. In case an amortization
    is inserted on a schedule, DCT cannot be calculated as the difference in days between the insertion date and the
    previous payment date.
    '''

    @dataclasses.dataclass
    class Bare:
        '''
        A minimum amortization entry.

        This class is useful to specify prepayments. In this case, only the date and the amortization percentage are
        sufficient.
        '''

        # Maximum value. Ver “http://stackoverflow.com/a/28082106”.
        MAX_VALUE: t.ClassVar[decimal.Decimal] = decimal.Decimal(decimal.MAX_EMAX)

        # Base field, the bare amortization date.
        date: datetime.date

        # Base field, the bare amortization nominal value.
        value: decimal.Decimal = _0

        # Extension field.
        dct_override: t.Optional[DctOverride] = None

    # Base field, the amortization date.
    date: datetime.date

    # Base field, the amortization percentage. FIXME: should never be negative.
    amortization_ratio: decimal.Decimal = _0

    # Base field, whether the interest will be paid or will accumulate for a subsequent date.
    amortizes_interest: bool = True

    # Extension field.
    price_level_adjustment: t.Optional[PriceLevelAdjustment] = None

    # Extension field.
    dct_override: t.Optional[DctOverride] = None

@dataclasses.dataclass
class Payment:
    '''
    An entry of a payment schedule.

    It represents a payment at a given date.

      • "no" is the payment number;

      • "date" is the date the payment should be done;

      • "raw" is the raw value of the payment;

      • "tax" is the revenue tax to collect;

      • "net" is the net value to be payed;

      • "gain" is the yielded profit;

      • "amort" is the amortized value;

      • "bal" is the current loan balance.
    '''

    no: int = 0

    date: datetime.date = datetime.date.min

    raw: decimal.Decimal = _0

    tax: decimal.Decimal = _0

    net: decimal.Decimal = _0

    gain: decimal.Decimal = _0

    amort: decimal.Decimal = _0

    bal: decimal.Decimal = _0

@dataclasses.dataclass
class DailyReturn:
    no: int = 0

    period: int = 0

    date: datetime.date = datetime.date.min

    value: decimal.Decimal = _0

    bal: decimal.Decimal = _0

    fixed_factor: decimal.Decimal = _1

    variable_factor: decimal.Decimal = _1

@dataclasses.dataclass
class PriceAdjustedPayment(Payment):
    '''An entry of a payment schedule, with price level adjustment (IPCA or IGPM).'''

    pla: decimal.Decimal = _0

@dataclasses.dataclass
class LatePayment(Payment):
    '''An entry of a payment schedule, with extra gain, penalty and fine values.'''

    # Taxa mensal de juros moratórios (a.m.). MLFR = Montly Late Fee Rate.
    FEE_RATE: t.ClassVar[decimal.Decimal] = _1

    # Taxa de multa.
    FINE_RATE: t.ClassVar[decimal.Decimal] = _1 + _1

    extra_gain: decimal.Decimal = _0

    penalty: decimal.Decimal = _0

    fine: decimal.Decimal = _0

@dataclasses.dataclass
class LatePriceAdjustedPayment(PriceAdjustedPayment):
    '''An entry of a price adjusted payment schedule, with extra gain, penalty and fine values.'''

    extra_gain: decimal.Decimal = _0

    penalty: decimal.Decimal = _0

    fine: decimal.Decimal = _0

@dataclasses.dataclass
class CalcDate:
    '''
    A calculation date for the "get_payments_table" function.

    When creating a payment table from an amortization schedule, the value specified in the date field of this class
    will establish where the calculations will stop. The returned table will only show payments up to that date.

    The runaway field, when true, dictates that the entire payments schedule should be returned.
    '''

    value: datetime.date

    runaway: bool = False
# }}}

# Public API. Variable index, and storage backend classes. {{{
class BackendError(Exception):
    pass

@dataclasses.dataclass
class DailyIndex:
    date: datetime.date = datetime.date.min

    value: decimal.Decimal = _0

@dataclasses.dataclass
class MonthlyIndex:
    date: datetime.date = datetime.date.min

    value: decimal.Decimal = _0

@dataclasses.dataclass
class RangedIndex:
    begin_date: datetime.date = datetime.date.min

    end_date: datetime.date = datetime.date.min

    value: decimal.Decimal = _0

class IndexStorageBackend:
    def get_cdi_indexes(self, begin: datetime.date, end: datetime.date) -> t.Iterable[DailyIndex]:
        '''
        Returns the list of CDI indexes between the begin and end date.

        The begin date is inclusive. The end date is exclusive.

        This method should project indexes in the future.
        '''

        raise NotImplementedError()

    def get_savings_indexes(self, begin: datetime.date, end: datetime.date) -> t.Iterable[RangedIndex]:
        '''
        Returns the list of Savings indexes between the begin and end date.

        This method should not project indexes in the future.
        '''

        raise NotImplementedError()

    def get_ipca_indexes(self, begin: datetime.date, end: datetime.date) -> t.Iterable[MonthlyIndex]:
        '''
        Returns the list of IPCA indexes between the begin and end date.

        This method should not project indexes in the future.
        '''

        raise NotImplementedError()

    def get_igpm_indexes(self, begin: datetime.date, end: datetime.date) -> t.Iterable[MonthlyIndex]:
        '''
        Returns the list of IGPM indexes between the begin and end date.

        This method should not project indexes in the future.
        '''

        raise NotImplementedError()

    @typeguard.typechecked
    def calculate_cdi_factor(self, begin: datetime.date, end: datetime.date, percentage: int = 100) -> types.SimpleNamespace:
        '''
        Calcula o fator de DI (CDI) a partir de um período.

        Os índices de correção CDI dos testes abaixo foram retirados do sítio do BACEN:

          https://www3.bcb.gov.br/CALCIDADAO/publico/exibirFormCorrecaoValores.do?method=exibirFormCorrecaoValores&aba=5

        >>> from math import isclose
        >>> from datetime import date
        >>>
        >>> bend = InMemoryBackend()
        >>>
        >>> idx1 = decimal.Decimal('1.10949606')
        >>> idx2 = bend.calculate_cdi_factor(date(2022, 1, 10), date(2022, 12, 1))
        >>> idx2.amount
        224
        >>> isclose(idx1, idx2.value, rel_tol=1e-8)
        True

        >>> idx1 = decimal.Decimal('1.06421360')
        >>> idx2 = bend.calculate_cdi_factor(date(2018, 1, 2), date(2019, 1, 2))
        >>> idx2.amount
        250
        >>> isclose(idx1, idx2.value, rel_tol=1e-8)
        True

        >>> idx1 = decimal.Decimal('1.20999561')
        >>> idx2 = bend.calculate_cdi_factor(date(2018, 1, 2), date(2022, 1, 3))
        >>> idx2.amount
        1005
        >>> isclose(idx1, idx2.value, rel_tol=1e-8)
        True

        Observe que não é possível escrever testes para índices projetados. Essa função vai pegar o último índice publicado
        pelo BACEN para estimar os futuros. Um novo índice pode ser publicado, alterando o resultado da computação do
        fator, e fazendo com que o teste falhe.
        '''

        if begin < end:
            pct = decimal.Decimal(percentage) / decimal.Decimal(100)
            end = end - datetime.timedelta(days=1)  # Último dia, sempre excludente.
            fac = _1
            cnt = 0

            for x in self.get_cdi_indexes(begin, end):
                fac = fac * (1 + pct * x.value / decimal.Decimal(100))

                cnt += 1

                _LOG.debug(x)

            return types.SimpleNamespace(value=fac, amount=cnt)

        return types.SimpleNamespace(value=_1, amount=0)

    @typeguard.typechecked
    def calculate_savings_factor(self, begin: datetime.date, end: datetime.date, percentage: int = 100) -> types.SimpleNamespace:
        '''Calcula o fator de Poupança a partir de um período.'''

        # Considera-se a data de aniversário das prestações com início nos dias 29, 30 e 31 como o dia 1° do mês seguinte.
        ini = begin if begin.day <= 28 else (begin + _MONTH).replace(day=1)
        pct = decimal.Decimal(percentage) / decimal.Decimal(100)
        fac = _1
        mem = []

        # Para cada mês, apenas um dos seus 28 índices será selecionado aqui. Lembrar que apenas nos primeiros 28 dias
        # do mês tem-se publicação de Poupança. Se a data "begin" cair após do dia 28, considera-se o dia primeiro do
        # mês subsequente. Considera como o índice do mês, M, aquele em que "M.begin_date.day" seja igual a
        # "begin.day".
        #
        for x in self.get_savings_indexes(ini, end):
            if ini.day == x.begin_date.day:
                fac = fac * (_1 + pct * x.value / decimal.Decimal(100))

                mem.append(x)

                _LOG.debug(x)

        return types.SimpleNamespace(value=fac, amount=len(mem))

    @typeguard.typechecked
    def calculate_ipca_factor(self, base: datetime.date, period: int, shift: _PL_SHIFT, ratio: decimal.Decimal = _1) -> decimal.Decimal:
        '''
        Calcula o fator de correção do IPCA.

        Toma como parâmetros a data base, o período, deslocamento e uma fração para a última taxa de correção.
        '''

        ini = base - _MONTH * t.get_args(_PL_SHIFT).index(shift)
        end = base + _MONTH * (period - 1) - _MONTH * t.get_args(_PL_SHIFT).index(shift)
        mem = list(self.get_ipca_indexes(ini, end))  # FIXME: try to avoid this conversion to list.
        fac = _1

        for i, x in enumerate(mem):
            exp = _1 if i != len(mem) - 1 else ratio

            fac = fac * (_1 + x.value / decimal.Decimal(100)) ** exp

        return fac

    @typeguard.typechecked
    def calculate_igpm_factor(self, base: datetime.date, period: int, shift: _PL_SHIFT, ratio: decimal.Decimal = _1) -> decimal.Decimal:
        '''Calcula o fator de correção do IGPM.'''

        raise NotImplementedError()

class InMemoryBackend(IndexStorageBackend):
    '''
    This backend simulates almost five years of BACEN API.

    Indexes for CDI, IPCA and Poupança, ranging from 2018-01-01 to 2022-11-15, are kept in memory.

    As this backend is mostly static, i.e., it does not update itself as new indexes are published, it isn't suited for
    production purposes.

    It is fast since all data sets for CDI, IPCA, and Poupança are kept in primary memory. But isn't particularly
    clever. For a given date range, no matter how small or large it is, the index fetching methods will always scan the
    entire data sets.
    '''

    _ignore_cdi = [
        datetime.date(2018, 1, 1),   datetime.date(2018, 2, 12),  datetime.date(2018, 2, 13),  datetime.date(2018, 3, 30),  # NOQA.
        datetime.date(2018, 5, 1),   datetime.date(2018, 5, 31),  datetime.date(2018, 9, 7),   datetime.date(2018, 10, 12), # NOQA.
        datetime.date(2018, 11, 2),  datetime.date(2018, 11, 15), datetime.date(2018, 12, 25), datetime.date(2019, 1, 1),   # NOQA.
        datetime.date(2019, 3, 4),   datetime.date(2019, 3, 5),   datetime.date(2019, 4, 19),  datetime.date(2019, 5, 1),   # NOQA.
        datetime.date(2019, 6, 20),  datetime.date(2019, 11, 15), datetime.date(2019, 12, 25), datetime.date(2020, 1, 1),   # NOQA.
        datetime.date(2020, 2, 24),  datetime.date(2020, 2, 25),  datetime.date(2020, 4, 10),  datetime.date(2020, 4, 21),  # NOQA.
        datetime.date(2020, 5, 1),   datetime.date(2020, 6, 11),  datetime.date(2020, 9, 7),   datetime.date(2020, 10, 12), # NOQA.
        datetime.date(2020, 11, 2),  datetime.date(2020, 12, 25), datetime.date(2021, 1, 1),   datetime.date(2021, 2, 15),  # NOQA.
        datetime.date(2021, 2, 16),  datetime.date(2021, 4, 2),   datetime.date(2021, 4, 21),  datetime.date(2021, 6, 3),   # NOQA.
        datetime.date(2021, 9, 7),   datetime.date(2021, 10, 12), datetime.date(2021, 11, 2),  datetime.date(2021, 11, 15), # NOQA.
        datetime.date(2022, 2, 28),  datetime.date(2022, 3, 1),   datetime.date(2022, 4, 15),  datetime.date(2022, 4, 21),  # NOQA.
        datetime.date(2022, 6, 16),  datetime.date(2022, 9, 7),   datetime.date(2022, 10, 12), datetime.date(2022, 11, 2),  # NOQA.
        datetime.date(2022, 11, 15), datetime.date(2023, 2, 20),  datetime.date(2023, 2, 21),  datetime.date(2023, 4, 7),   # NOQA.
        datetime.date(2023, 4, 21),  datetime.date(2023, 5, 1),   datetime.date(2023, 6, 8)                                 # NOQA.
    ]

    # A repository of CDI indexes.
    _registry_cdi = [
        (datetime.date(2017, 12, 29), datetime.date(2018, 2, 7),   decimal.Decimal('0.026444')),  # NOQA.
        (datetime.date(2018, 2, 8),   datetime.date(2018, 3, 21),  decimal.Decimal('0.025515')),  # NOQA.
        (datetime.date(2018, 3, 22),  datetime.date(2018, 9, 28),  decimal.Decimal('0.024583')),  # NOQA.
        (datetime.date(2018, 10, 1),  datetime.date(2019, 7, 31),  decimal.Decimal('0.024620')),  # NOQA.
        (datetime.date(2019, 8, 1),   datetime.date(2019, 9, 18),  decimal.Decimal('0.022751')),  # NOQA.
        (datetime.date(2019, 9, 19),  datetime.date(2019, 10, 30), decimal.Decimal('0.020872')),  # NOQA.
        (datetime.date(2019, 10, 31), datetime.date(2019, 12, 11), decimal.Decimal('0.018985')),  # NOQA.
        (datetime.date(2019, 12, 12), datetime.date(2020, 2, 5),   decimal.Decimal('0.017089')),  # NOQA.
        (datetime.date(2020, 2, 6),   datetime.date(2020, 3, 18),  decimal.Decimal('0.016137')),  # NOQA.
        (datetime.date(2020, 3, 19),  datetime.date(2020, 5, 6),   decimal.Decimal('0.014227')),  # NOQA.
        (datetime.date(2020, 5, 7),   datetime.date(2020, 6, 17),  decimal.Decimal('0.011345')),  # NOQA.
        (datetime.date(2020, 6, 18),  datetime.date(2020, 8, 5),   decimal.Decimal('0.008442')),  # NOQA.
        (datetime.date(2020, 8, 6),   datetime.date(2021, 3, 17),  decimal.Decimal('0.007469')),  # NOQA.
        (datetime.date(2021, 3, 18),  datetime.date(2021, 5, 5),   decimal.Decimal('0.010379')),  # NOQA.
        (datetime.date(2021, 5, 6),   datetime.date(2021, 6, 16),  decimal.Decimal('0.013269')),  # NOQA.
        (datetime.date(2021, 6, 17),  datetime.date(2021, 8, 4),   decimal.Decimal('0.016137')),  # NOQA.
        (datetime.date(2021, 8, 5),   datetime.date(2021, 9, 22),  decimal.Decimal('0.019930')),  # NOQA.
        (datetime.date(2021, 9, 23),  datetime.date(2021, 10, 27), decimal.Decimal('0.023687')),  # NOQA.
        (datetime.date(2021, 10, 28), datetime.date(2021, 12, 8),  decimal.Decimal('0.029256')),  # NOQA.
        (datetime.date(2021, 12, 9),  datetime.date(2022, 2, 2),   decimal.Decimal('0.034749')),  # NOQA.
        (datetime.date(2022, 2, 3),   datetime.date(2022, 3, 16),  decimal.Decimal('0.040168')),  # NOQA.
        (datetime.date(2022, 3, 17),  datetime.date(2022, 5, 4),   decimal.Decimal('0.043739')),  # NOQA.
        (datetime.date(2022, 5, 5),   datetime.date(2022, 6, 15),  decimal.Decimal('0.047279')),  # NOQA.
        (datetime.date(2022, 6, 17),  datetime.date(2022, 8, 3),   decimal.Decimal('0.049037')),  # NOQA.
        (datetime.date(2022, 8, 4),   datetime.date(2022, 11, 14), decimal.Decimal('0.050788'))   # NOQA.
    ]

    # A repository of IPCA indexes.
    _registry_ipca = [
        (datetime.date(2018, 1, 1),  decimal.Decimal('0.29')),  (datetime.date(2018, 2, 1),  decimal.Decimal('0.32')),  # NOQA.
        (datetime.date(2018, 3, 1),  decimal.Decimal('0.09')),  (datetime.date(2018, 4, 1),  decimal.Decimal('0.22')),  # NOQA.
        (datetime.date(2018, 5, 1),  decimal.Decimal('0.40')),  (datetime.date(2018, 6, 1),  decimal.Decimal('1.26')),  # NOQA.
        (datetime.date(2018, 7, 1),  decimal.Decimal('0.33')),  (datetime.date(2018, 8, 1),  decimal.Decimal('-0.09')), # NOQA.
        (datetime.date(2018, 9, 1),  decimal.Decimal('0.48')),  (datetime.date(2018, 10, 1), decimal.Decimal('0.45')),  # NOQA.
        (datetime.date(2018, 11, 1), decimal.Decimal('-0.21')), (datetime.date(2018, 12, 1), decimal.Decimal('0.15')),  # NOQA.
        (datetime.date(2019, 1, 1),  decimal.Decimal('0.32')),  (datetime.date(2019, 2, 1),  decimal.Decimal('0.43')),  # NOQA.
        (datetime.date(2019, 3, 1),  decimal.Decimal('0.75')),  (datetime.date(2019, 4, 1),  decimal.Decimal('0.57')),  # NOQA.
        (datetime.date(2019, 5, 1),  decimal.Decimal('0.13')),  (datetime.date(2019, 6, 1),  decimal.Decimal('0.01')),  # NOQA.
        (datetime.date(2019, 7, 1),  decimal.Decimal('0.19')),  (datetime.date(2019, 8, 1),  decimal.Decimal('0.11')),  # NOQA.
        (datetime.date(2019, 9, 1),  decimal.Decimal('-0.04')), (datetime.date(2019, 10, 1), decimal.Decimal('0.10')),  # NOQA.
        (datetime.date(2019, 11, 1), decimal.Decimal('0.51')),  (datetime.date(2019, 12, 1), decimal.Decimal('1.15')),  # NOQA.
        (datetime.date(2020, 1, 1),  decimal.Decimal('0.21')),  (datetime.date(2020, 2, 1),  decimal.Decimal('0.25')),  # NOQA.
        (datetime.date(2020, 3, 1),  decimal.Decimal('0.07')),  (datetime.date(2020, 4, 1),  decimal.Decimal('-0.31')), # NOQA.
        (datetime.date(2020, 5, 1),  decimal.Decimal('-0.38')), (datetime.date(2020, 6, 1),  decimal.Decimal('0.26')),  # NOQA.
        (datetime.date(2020, 7, 1),  decimal.Decimal('0.36')),  (datetime.date(2020, 8, 1),  decimal.Decimal('0.24')),  # NOQA.
        (datetime.date(2020, 9, 1),  decimal.Decimal('0.64')),  (datetime.date(2020, 10, 1), decimal.Decimal('0.86')),  # NOQA.
        (datetime.date(2020, 11, 1), decimal.Decimal('0.89')),  (datetime.date(2020, 12, 1), decimal.Decimal('1.35')),  # NOQA.
        (datetime.date(2021, 1, 1),  decimal.Decimal('0.25')),  (datetime.date(2021, 2, 1),  decimal.Decimal('0.86')),  # NOQA.
        (datetime.date(2021, 3, 1),  decimal.Decimal('0.93')),  (datetime.date(2021, 4, 1),  decimal.Decimal('0.31')),  # NOQA.
        (datetime.date(2021, 5, 1),  decimal.Decimal('0.83')),  (datetime.date(2021, 6, 1),  decimal.Decimal('0.53')),  # NOQA.
        (datetime.date(2021, 7, 1),  decimal.Decimal('0.96')),  (datetime.date(2021, 8, 1),  decimal.Decimal('0.87')),  # NOQA.
        (datetime.date(2021, 9, 1),  decimal.Decimal('1.16')),  (datetime.date(2021, 10, 1), decimal.Decimal('1.25')),  # NOQA.
        (datetime.date(2021, 11, 1), decimal.Decimal('0.95')),  (datetime.date(2021, 12, 1), decimal.Decimal('0.73')),  # NOQA.
        (datetime.date(2022, 1, 1),  decimal.Decimal('0.54')),  (datetime.date(2022, 2, 1),  decimal.Decimal('1.01')),  # NOQA.
        (datetime.date(2022, 3, 1),  decimal.Decimal('1.62')),  (datetime.date(2022, 4, 1),  decimal.Decimal('1.06')),  # NOQA.
        (datetime.date(2022, 5, 1),  decimal.Decimal('0.47')),  (datetime.date(2022, 6, 1),  decimal.Decimal('0.67')),  # NOQA.
        (datetime.date(2022, 7, 1),  decimal.Decimal('-0.68')), (datetime.date(2022, 8, 1),  decimal.Decimal('-0.36')), # NOQA.
        (datetime.date(2022, 9, 1),  decimal.Decimal('-0.29')), (datetime.date(2022, 10, 1), decimal.Decimal('0.59')),  # NOQA.
        (datetime.date(2022, 11, 1), decimal.Decimal('0.41'))
    ]

    # A repository of Poupança indexes.
    _registry_savs = [
        (datetime.date(2018, 1, 1),  [decimal.Decimal(x) for x in ['0.3994'] * 28]),                                  # NOQA.
        (datetime.date(2018, 2, 1),  [decimal.Decimal(x) for x in ['0.3994'] * 7 + ['0.3855'] * 21]),                 # NOQA.
        (datetime.date(2018, 3, 1),  [decimal.Decimal(x) for x in ['0.3855'] * 21 + ['0.3715'] * 7]),                 # NOQA.
        (datetime.date(2018, 4, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2018, 5, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2018, 6, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2018, 7, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2018, 8, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2018, 9, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2018, 10, 1), [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2018, 11, 1), [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2018, 12, 1), [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2019, 1, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2019, 2, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2019, 3, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2019, 4, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2019, 5, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2019, 6, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2019, 7, 1),  [decimal.Decimal(x) for x in ['0.3715'] * 28]),                                  # NOQA.
        (datetime.date(2019, 8, 1),  [decimal.Decimal(x) for x in ['0.3434'] * 28]),                                  # NOQA.
        (datetime.date(2019, 9, 1),  [decimal.Decimal(x) for x in ['0.3434'] * 18 + ['0.3153'] * 10]),                # NOQA.
        (datetime.date(2019, 10, 1), [decimal.Decimal(x) for x in ['0.3153'] * 28]),                                  # NOQA.
        (datetime.date(2019, 11, 1), [decimal.Decimal(x) for x in ['0.2871'] * 28]),                                  # NOQA.
        (datetime.date(2019, 12, 1), [decimal.Decimal(x) for x in ['0.2871'] * 11 + ['0.2588'] * 17]),
        (datetime.date(2020, 1, 1),  [decimal.Decimal(x) for x in ['0.2588'] * 28]),                                  # NOQA.
        (datetime.date(2020, 2, 1),  [decimal.Decimal(x) for x in ['0.2588'] * 5 + ['0.2446'] * 23]),                 # NOQA.
        (datetime.date(2020, 3, 1),  [decimal.Decimal(x) for x in ['0.2446'] * 18 + ['0.2162'] * 10]),                # NOQA.
        (datetime.date(2020, 4, 1),  [decimal.Decimal(x) for x in ['0.2162'] * 28]),                                  # NOQA.
        (datetime.date(2020, 5, 1),  [decimal.Decimal(x) for x in ['0.2162'] * 6 + ['0.1733'] * 22]),                 # NOQA.
        (datetime.date(2020, 6, 1),  [decimal.Decimal(x) for x in ['0.1733'] * 17 + ['0.1303'] * 21]),                # NOQA.
        (datetime.date(2020, 7, 1),  [decimal.Decimal(x) for x in ['0.1303'] * 28]),                                  # NOQA.
        (datetime.date(2020, 8, 1),  [decimal.Decimal(x) for x in ['0.1303'] * 5 + ['0.1159'] * 23]),                 # NOQA.
        (datetime.date(2020, 9, 1),  [decimal.Decimal(x) for x in ['0.1159'] * 28]),                                  # NOQA.
        (datetime.date(2020, 10, 1), [decimal.Decimal(x) for x in ['0.1159'] * 28]),                                  # NOQA.
        (datetime.date(2020, 11, 1), [decimal.Decimal(x) for x in ['0.1159'] * 28]),                                  # NOQA.
        (datetime.date(2020, 12, 1), [decimal.Decimal(x) for x in ['0.1159'] * 28]),                                  # NOQA.
        (datetime.date(2021, 1, 1),  [decimal.Decimal(x) for x in ['0.1159'] * 28]),                                  # NOQA.
        (datetime.date(2021, 2, 1),  [decimal.Decimal(x) for x in ['0.1159'] * 28]),                                  # NOQA.
        (datetime.date(2021, 3, 1),  [decimal.Decimal(x) for x in ['0.1159'] * 17 + ['0.1590'] * 11]),                # NOQA.
        (datetime.date(2021, 4, 1),  [decimal.Decimal(x) for x in ['0.1590'] * 28]),                                  # NOQA.
        (datetime.date(2021, 5, 1),  [decimal.Decimal(x) for x in ['0.1590'] * 5 + ['0.2019'] * 23]),                 # NOQA.
        (datetime.date(2021, 6, 1),  [decimal.Decimal(x) for x in ['0.2019'] * 16 + ['0.2446'] * 12]),                # NOQA.
        (datetime.date(2021, 7, 1),  [decimal.Decimal(x) for x in ['0.2446'] * 28]),                                  # NOQA.
        (datetime.date(2021, 8, 1),  [decimal.Decimal(x) for x in ['0.2446'] * 4 + ['0.3012'] * 24]),                 # NOQA.
        (datetime.date(2021, 9, 1),  [decimal.Decimal(x) for x in ['0.3012'] * 22 + ['0.3575'] * 6]),                 # NOQA.
        (datetime.date(2021, 10, 1), [decimal.Decimal(x) for x in ['0.3575'] * 27 + ['0.4412']]),                     # NOQA.
        (datetime.date(2021, 11, 1), [decimal.Decimal(x) for x in ['0.4412'] * 15 + [
                                                                   '0.4556', '0.4578', '0.4586', '0.4412', '0.4412',  # NOQA.
                                                                   '0.4412', '0.4570', '0.4583', '0.4607', '0.4620',
                                                                   '0.4412', '0.4412', '0.4412']]),
        (datetime.date(2021, 12, 1), [decimal.Decimal(x) for x in ['0.4902', '0.4739', '0.4572', '0.4626', '0.4890',
                                                                   '0.5154', '0.5249', '0.5237', '0.5655', '0.5438',
                                                                   '0.5471', '0.5732', '0.5992', '0.6029', '0.6042',
                                                                   '0.5839', '0.5500', '0.5539', '0.5900', '0.6162',
                                                                   '0.6201', '0.6147', '0.5910', '0.5691', '0.5739',
                                                                   '0.6002', '0.6265', '0.6319']]),
        (datetime.date(2022, 1, 1),  [decimal.Decimal(x) for x in ['0.5608', '0.5872', '0.6138', '0.6158', '0.6146',  # NOQA.
                                                                   '0.5908', '0.5660', '0.5677', '0.5946', '0.6215',
                                                                   '0.6249', '0.6255', '0.6000', '0.5751', '0.5764',
                                                                   '0.6036', '0.6310', '0.6324', '0.6340', '0.6107',
                                                                   '0.5845', '0.5877', '0.6156', '0.6435', '0.6443',
                                                                   '0.6371', '0.6119', '0.5480']]),
        (datetime.date(2022, 2, 1),  [decimal.Decimal(x) for x in ['0.5000'] * 28]),  # NOQA.
        (datetime.date(2022, 3, 1),  [decimal.Decimal(x) for x in ['0.5976', '0.6304', '0.5997', '0.5673', '0.5779',  # NOQA.
                                                                   '0.6017', '0.6355', '0.6393', '0.6422', '0.6129',
                                                                   '0.5855', '0.5938', '0.6274', '0.6513', '0.6559',
                                                                   '0.6260', '0.6063', '0.5748', '0.5762', '0.6095',
                                                                   '0.6329', '0.6021', '0.6046', '0.5839', '0.5512',
                                                                   '0.5524', '0.5856', '0.6089']]),
        (datetime.date(2022, 4, 1),  [decimal.Decimal(x) for x in ['0.5558', '0.5243', '0.5577', '0.5809', '0.5805',  # NOQA.
                                                                   '0.5822', '0.5842', '0.5614', '0.5325', '0.5661',
                                                                   '0.5898', '0.5924', '0.5937', '0.5933', '0.5598',
                                                                   '0.5598', '0.5938', '0.6277', '0.6286', '0.6308',
                                                                   '0.6309', '0.6309', '0.5973', '0.6215', '0.6557',
                                                                   '0.6546', '0.6576', '0.6617']]),
        (datetime.date(2022, 5, 1),  [decimal.Decimal(x) for x in ['0.6671', '0.6919', '0.6914', '0.6947', '0.6646',  # NOQA.
                                                                   '0.6408', '0.6411', '0.6660', '0.7011', '0.7013',
                                                                   '0.7033', '0.6672', '0.6414', '0.6425', '0.6674',
                                                                   '0.7025', '0.6692', '0.6697', '0.6441', '0.6084',
                                                                   '0.6067', '0.6418', '0.6667', '0.6724', '0.6719',
                                                                   '0.6462', '0.6112', '0.6118']]),
        (datetime.date(2022, 6, 1),  [decimal.Decimal(x) for x in ['0.6491', '0.6519', '0.6162', '0.5828', '0.6195',  # NOQA.
                                                                   '0.6462', '0.6491', '0.6509', '0.6520', '0.6260',
                                                                   '0.5950', '0.6218', '0.6588', '0.6602', '0.6652',
                                                                   '0.6643', '0.6643', '0.6279', '0.6648', '0.6917',
                                                                   '0.6933', '0.6924', '0.6929', '0.6676', '0.6332',
                                                                   '0.6701', '0.6972', '0.6972']]),
        (datetime.date(2022, 7, 1),  [decimal.Decimal(x) for x in ['0.6639', '0.6643', '0.7013', '0.7284', '0.7281',  # NOQA.
                                                                   '0.7278', '0.7008', '0.6642', '0.6659', '0.7031',
                                                                   '0.7303', '0.7307', '0.7324', '0.7058', '0.6696',
                                                                   '0.6710', '0.7083', '0.7358', '0.7373', '0.7372',
                                                                   '0.7100', '0.6724', '0.6730', '0.7105', '0.7381',
                                                                   '0.7385', '0.7386', '0.7132']]),
        (datetime.date(2022, 8, 1),  [decimal.Decimal(x) for x in ['0.7421', '0.7420', '0.7432', '0.7083', '0.6804',  # NOQA.
                                                                   '0.6810', '0.7088', '0.7088', '0.7075', '0.7075',
                                                                   '0.6798', '0.6521', '0.6526', '0.6803', '0.7082',
                                                                   '0.7084', '0.7086', '0.6793', '0.6524', '0.6524',
                                                                   '0.6801', '0.7079', '0.7087', '0.7087', '0.6809',
                                                                   '0.6527', '0.6430', '0.6808']]),
        (datetime.date(2022, 9, 1),  [decimal.Decimal(x) for x in ['0.6814', '0.6432', '0.6152', '0.6430', '0.6809',  # NOQA.
                                                                   '0.6809', '0.6817', '0.7097', '0.6818', '0.6440',
                                                                   '0.6819', '0.7097', '0.6819', '0.6830', '0.6835',
                                                                   '0.6470', '0.6198', '0.6477', '0.6859', '0.6850',
                                                                   '0.6845', '0.6824', '0.6512', '0.6142', '0.6520',
                                                                   '0.6797', '0.6779', '0.6777']]),
        (datetime.date(2022, 10, 1), [decimal.Decimal(x) for x in ['0.6501', '0.6778', '0.6778', '0.6789', '0.6796',
                                                                   '0.6520', '0.6136', '0.6137', '0.6514', '0.6790',
                                                                   '0.6811', '0.6798', '0.6798', '0.6516', '0.6515',
                                                                   '0.6515', '0.6791', '0.6789', '0.6794', '0.6513',
                                                                   '0.6143', '0.6147', '0.6524', '0.6801', '0.6794',
                                                                   '0.6791', '0.6516', '0.6136']]),
        (datetime.date(2022, 11, 1), [decimal.Decimal(x) for x in ['0.6515', '0.6515', '0.6792', '0.6519', '0.6139',
                                                                   '0.6516', '0.6793', '0.6799', '0.6801', '0.6796'] + ['0.6448'] * 18])  # As 17 taxas finais são estimadas.
    ]

    @typeguard.typechecked
    def get_cdi_indexes(self, begin: datetime.date, end: datetime.date) -> t.Iterable[DailyIndex]:
        if begin >= self._registry_cdi[0][0] and end >= begin:
            save = self._registry_cdi[0][2]

            for d0, d1, value in self._registry_cdi:
                d = d0

                while d <= d1:
                    if d <= end:
                        save = value

                    if begin <= d <= end and d.weekday() < 5 and d not in self._ignore_cdi:
                        yield DailyIndex(date=d, value=value)

                    d += datetime.timedelta(days=1)

            while d <= end:
                if d >= begin and d.weekday() < 5 and d not in self._ignore_cdi:
                    yield DailyIndex(date=d, value=save)

                d += datetime.timedelta(days=1)

        elif begin >= self._registry_cdi[0][0]:
            raise ValueError('the initial date must be greater than, or equal to, the end date')

        else:
            raise ValueError('this backend cannot provide CDI indexes prior to 2018-01-01')

    # FIXME: this method attempts to simulate the behaviour of the BACEN API. But the API is flawed. For a monthly
    # index, It can't even properly represent months, using their first days to represent them. For example,
    # "2018-01-01" represents January of 2018. Why not use simply "2018-01"???
    #
    # I do not think that fincore data structures, like MonthlyIndex, should expose these flaws internally. It should
    # be designed to best suit its use by this module.
    #
    @typeguard.typechecked
    def get_ipca_indexes(self, begin: datetime.date, end: datetime.date) -> t.Iterable[MonthlyIndex]:
        for month, value in self._registry_ipca:
            if begin <= month <= end:
                yield MonthlyIndex(date=month, value=value)

        while month < end:
            month += _MONTH

            yield MonthlyIndex(date=month, value=_0)

    # FIXME: this method simulates the behaviour of the BACEN API. But the API is pretty dumb. It returns redundant data,
    # like "2018-01-01" to represent January of 2018.
    #
    @typeguard.typechecked
    def get_savings_indexes(self, begin: datetime.date, end: datetime.date) -> t.Iterable[RangedIndex]:
        for d0, values in self._registry_savs:
            i, d = 0, d0

            while i < 28:
                if begin <= d <= end:
                    yield RangedIndex(begin_date=d, end_date=d + _MONTH, value=values[i])

                d += datetime.timedelta(days=1)
                i += 1

@dataclasses.dataclass(frozen=True, eq=True)
class VariableIndex:
    code: t.Union[_VR_INDEX, _PL_INDEX] = 'CDI'

    percentage: int = 100

    backend: IndexStorageBackend = dataclasses.field(default=InMemoryBackend(), compare=False)
# }}}

# !!! Rotina nuclear: "get_payments_table". !!! {{{
@typeguard.typechecked
def get_payments_table(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    amortizations: t.List[Amortization | Amortization.Bare], *,
    vir: t.Optional[VariableIndex] = None,
    capitalisation: _CAPITALISATION = '360',
    calc_date: t.Optional[CalcDate] = None,
    tax_exempt: t.Optional[bool] = False,
    gain_output: _GAIN_OUTPUT_MODE = 'current'
) -> t.Iterable[Payment]:
    '''
    Gera uma tabela de pagamentos para um determinado empréstimo.

    Para saber como invocar essa função, tome como base a frase abaixo.

      “ Retorne os pagamentos um empréstimo em um valor V, a uma taxa anual TA, nas datas D. ”

    Dessa elaboração saem os três parâmetros obrigatórios e posicionais dessa rotina:

      • "principal", é o valor principal do empréstimo, ou V.

      • "apy", é a taxa nominal anual de spread TA (annual percentage yield).

      • "amortizations", que é uma lista contendo as datas D em que amortizações devem ser realizadas.

    Os demais parâmetros são associativos.

      • "vir", é um índice variável, que pode ser de juros: CDI, ou Poupança; ou de correção monetária: IPCA, ou IGPM.
        Quando informado, os pagamentos retornados pela rotina vão incorrer, além da taxa fixa, "apy", de uma taxa
        variável que será computada de acordo com o valor do índice no período. Deve ser uma instância de
        VariableIndex. Se omitido, a taxa de juros será fixa.

      • "capitalisation", configura uma dentre quatro formas de composição dos juros.

        – "360" é a capitalização de juros diária em ano tem 360 dias corridos. Usada em operações pré-fixadas com
          modalidade Bullet.

        – "30/360" é a capitalização mensal em ano de 12 meses com 30 dias corridos. Usada em operações pré-fixadas com
          modalidade Juros mensais, Price, ou Livre.

        – "252" é a capitalização de juros em ano de 252 dias úteis. Usada em operações pós-fixadas CDI com quaisquer
          modalidade Bullet, Juros mensais, ou Livre.

        – "365" é a capitalização de juros em ano de 365 dias corridos. Esse modo não deve ser usado. Serve apenas
          para emular operações Bullet obsoletas, que adotaram essa forma de composição no passado. O Fincore vai
          emitir um aviso se esse modo for solicitado.

      • "calc_date", é uma data de corte para o cálculo dos pagamentos. Pagamentos posteriores à data informada não serão
        emitidos. Útil para saber a posição de um empréstimo em uma determinada data. Evita que pagamentos desnecessários
        sejam calculados.

      • "tax_exempt", quando verdadeiro, indica que os pagamentos são isentos de imposto de renda.

    • "gain_output", é o modo de saída para os juros.

        – "current" é o modo padrão. Faz com que, para cada pagamento P, apenas os juros incorridos no período entre o
          pagamento anterior e o atual sejam retornados.

        – "settled" faz com que, para cada pagamento P, os juros a serem pagos pelo devedor sejam retornados. Lembrar que
          os juros a serem pagos não são necessariamente iguais aos juros incorridos no período. Se houver carência, um
          determinado pagamento pode não ter acerto de juros. Ou um pagamento posterior a um período de carência pode
          apresentar um componente de juros acumulados, além do que incorreu no período.

        – "deferred" faz com que o juro acumulado no período atual, somado ao acumulado de períodos anteriores, seja
          retornado.

    Retorna uma lista em que cada objeto P contém as informações de um pagamento: sua data, o valor bruto a ser pago, o
    imposto, o valor líquido, o valor de amortização do principal, o valor dos juros etc. É uma instância de Payment,
    para operações sem correção monetária; ou de PriceAdjustedPayment, para operações corrigidas. Vide documentação
    dessas classes para maiores detalhes sobre essas estruturas de dados.
    '''

    def calc_balance() -> decimal.Decimal:
        val = principal * f_c + regs.interest.accrued - regs.principal.amortized.total * f_c - regs.interest.settled.total

        return t.cast(decimal.Decimal, val)

    # Primeiro gerador para valores de principal.
    #
    #  • "principal.amortization_ratio.current", é o percentual de amortização do período corrente.
    #  • "principal.amortized.current", é o valor amortizado no período corrente.
    #  • "principal.amortized.total", é o valor amortizado total (período corrente somado aos passados).
    #
    def track_principal_1() -> t.Generator[None, decimal.Decimal | None, None]:
        while True:
            ratio = yield

            # Se o percentual de amortização atual somado ao acumulado ultrapassar 100%, um reajuste deve ser feito.
            if regs.principal.amortization_ratio.current + ratio > _1:
                ratio = _1 - regs.principal.amortization_ratio.current

            if ratio:
                regs.principal.amortization_ratio.current += ratio
                regs.principal.amortized = types.SimpleNamespace(current=ratio * principal, total=regs.principal.amortization_ratio.current * principal)

            else:
                regs.principal.amortized = types.SimpleNamespace(current=_0, total=regs.principal.amortized.total)

    # Segundo gerador para valores de principal.
    #
    #  • "principal.amortization_ratio.regular", é o percentual de amortização regular acumulado (período corrente somado a passados)
    #
    def track_principal_2() -> t.Generator[None, decimal.Decimal | None, None]:
        while True:
            ratio = yield

            # Se o percentual de amortização regular somado ao acumulado ultrapassar 100%, um reajuste deve ser feito.
            if regs.principal.amortization_ratio.regular + ratio > _1:
                ratio = _1 - regs.principal.amortization_ratio.regular

            if ratio:
                regs.principal.amortization_ratio.regular += ratio

    # Gerador para valores de juros.
    #
    #   • "interest.current" são os juros incorridos (produzidos) no período corrente.
    #   • "interest.accrued" é o total de juros acumulado desde o dia zero do cronograma de pagamentos.
    #   • "interest.deferred" é o total de juros em aberto de períodos passados.
    #
    def track_interest_1() -> t.Generator[None, decimal.Decimal | None, None]:
        while True:
            regs.interest.current = yield
            regs.interest.accrued += regs.interest.current
            regs.interest.deferred = regs.interest.accrued - (regs.interest.current + regs.interest.settled.total)

    # Gerador para valores de juros acertados entre devedor e credor.
    #
    #   • "interest.settled.current" são os juros acertados no período corrente.
    #   • "interest.settled.total" é o total de juros acertados desde o dia zero do cronograma de pagamentos.
    #
    def track_interest_2() -> t.Generator[None, decimal.Decimal | None, None]:
        while True:
            regs.interest.settled = types.SimpleNamespace(current=(yield), total=regs.interest.settled.total)
            regs.interest.settled.total += regs.interest.settled.current

    # A. Valida e prepara para execução.
    gens = types.SimpleNamespace()
    regs = types.SimpleNamespace()
    aux = _0

    if principal == _0:
        return

    if principal < _CENTI:
        raise ValueError('principal value should be at least 0.01')

    if len(amortizations) < 2:
        raise ValueError('at least two amortizations are required: the start of the schedule, and its end')

    elif not vir and capitalisation == '252':
        raise ValueError('fixed interest rates should not use the 252 working days capitalisation')

    elif vir and vir.code == 'CDI' and capitalisation != '252':
        raise ValueError('CDI should use the 252 working days capitalisation')

    for i, x in enumerate(amortizations):
        if type(x) is Amortization:
            aux += x.amortization_ratio

        if vir and vir.code not in t.get_args(_PL_INDEX) and type(x) is Amortization and x.price_level_adjustment:
            raise TypeError(f"amortization {i} has price level adjustment, but either a variable index wasn't provided or it isn't IPCA nor IGPM")

        elif aux > _1 and not math.isclose(aux, _1):
            raise ValueError('the accumulated percentage of the amortizations overflows 1.0')

    if not math.isclose(aux, _1):
        raise ValueError('the accumulated percentage of the amortizations does not reach 1.0')

    if calc_date is None:
        calc_date = CalcDate(value=amortizations[-1].date, runaway=False)

    # Registradores.
    regs.interest = types.SimpleNamespace(current=_0, accrued=_0, settled=types.SimpleNamespace(current=_0, total=_0), deferred=_0)
    regs.principal = types.SimpleNamespace(amortization_ratio=types.SimpleNamespace(current=_0, regular=_0), amortized=types.SimpleNamespace(current=_0, total=_0))

    # Control, create generators.
    gens.interest_tracker_1 = track_interest_1()
    gens.interest_tracker_2 = track_interest_2()
    gens.principal_tracker_1 = track_principal_1()
    gens.principal_tracker_2 = track_principal_2()

    # Control, start generators.
    gens.principal_tracker_1.send(None)
    gens.principal_tracker_2.send(None)
    gens.interest_tracker_1.send(None)
    gens.interest_tracker_2.send(None)

    # B. Executa.
    for num, (ent0, ent1) in enumerate(itertools.pairwise(amortizations), 1):
        due = min(calc_date.value, ent1.date)
        f_s = f_c = _1

        # Fase B.0, FZA, ou Fase Zille-Anna.
        #
        #  • Calcula FS (fator de spread) para índice pré-fixado; e ambos FS e FC para índice de correção.
        #  • Calcula FS para índice pós-fixado (CDI, Poupança etc). Nesse caso não há correção.
        #
        if ent0.date < calc_date.value or ent1.date <= calc_date.value:
            if not vir and capitalisation == '360':  # Bullet.
                f_s = calculate_interest_factor(apy, decimal.Decimal((due - ent0.date).days) / decimal.Decimal(360))

            elif not vir and capitalisation == '365':  # Bullet in legacy mode.
                f_s = calculate_interest_factor(apy, decimal.Decimal((due - ent0.date).days) / decimal.Decimal(365))

            elif not vir and capitalisation == '30/360':  # Juros mensais, Price, Livre.
                dcp = (due - ent0.date).days
                dct = (ent1.date - ent0.date).days

                # Exclusivamente para a primeira data de aniversário o "DCT" será considerado como a diferença em
                # dias corridos entre o dia 24 anterior e o dia 24 posterior à data de integralização (início do
                # rendimento).
                #
                if ent1.dct_override and num == 1:
                    dct = _diff_surrounding_dates(ent0.date, 24)

                # Quando existirem entradas extraordinárias de adiantamento no plano, o "DCT" será calculado
                # utilizando as datas do fluxo regular.
                #
                elif ent1.dct_override:
                    dct = (ent1.dct_override.date_to - ent1.dct_override.date_from).days

                    if ent1.dct_override.predates_first_amortization:
                        dct = _diff_surrounding_dates(ent1.dct_override.date_from, 24)

                if ent0.dct_override:
                    dct = (ent1.date - ent0.dct_override.date_from).days

                    if ent0.dct_override.predates_first_amortization:
                        dct = _diff_surrounding_dates(ent0.dct_override.date_from, 24)

                f_s = calculate_interest_factor(apy, decimal.Decimal(dcp) / (12 * decimal.Decimal(dct)))

            elif vir and vir.code == 'CDI' and capitalisation == '252':  # Bullet, Juros mensais, Livre.
                f_v = vir.backend.calculate_cdi_factor(ent0.date, due, vir.percentage)  # Taxa (ou fator) variável, FV.
                f_s = calculate_interest_factor(apy, decimal.Decimal(f_v.amount) / decimal.Decimal(252)) * f_v.value

            elif vir and vir.code == 'Poupança' and capitalisation == '360':  # Poupança só suportada em Bullet.
                f_v = vir.backend.calculate_savings_factor(ent0.date, due, vir.percentage)  # Taxa (ou fator) variável, FV.
                f_s = calculate_interest_factor(apy, decimal.Decimal((due - ent0.date).days) / decimal.Decimal(360)) * f_v.value

            elif vir and vir.code == 'IPCA' and capitalisation == '360':  # Bullet.
                f_s = calculate_interest_factor(apy, decimal.Decimal((due - ent0.date).days) / decimal.Decimal(360))

                if type(ent1) is Amortization and ent1.price_level_adjustment:
                    kw1a: t.Dict[str, t.Any] = {}

                    kw1a['base'] = ent1.price_level_adjustment.base_date
                    kw1a['period'] = ent1.price_level_adjustment.period
                    kw1a['shift'] = ent1.price_level_adjustment.shift
                    kw1a['ratio'] = _1  # Ajuste para a última taxa de correção.

                    # Trava o fator de correção. O fator mínimo é um, ou seja, o valor da correção tem que ser positivo.
                    f_c = max(vir.backend.calculate_ipca_factor(**kw1a), _1)

                # Na antecipação a correção monetária tem que ser paga ("ent1" nem tem o atributo "price_level_adjustment").
                elif type(ent1) is Amortization.Bare:
                    kw1b: t.Dict[str, t.Any] = {}

                    kw1b['base'] = amortizations[0].date.replace(day=1)
                    kw1b['period'] = _delta_months(ent1.date, amortizations[0].date)
                    kw1b['shift'] = 'M-1'  # FIXME.
                    kw1b['ratio'] = _1  # Ajuste para a última taxa de correção.

                    # Trava o fator de correção. O fator mínimo é um, ou seja, o valor da correção tem que ser positivo.
                    f_c = max(vir.backend.calculate_ipca_factor(**kw1b), _1)

            elif vir and vir.code == 'IPCA' and capitalisation == '30/360':  # Juros mensais e Livre.
                dcp = (due - ent0.date).days
                dct = (ent1.date - ent0.date).days

                # Exclusivamente para a primeira data de aniversário o "DCT" será considerado como a diferença em
                # dias corridos entre o dia 24 anterior e o dia 24 posterior à data de integralização (início do
                # rendimento).
                #
                if ent1.dct_override and num == 1:
                    dct = _diff_surrounding_dates(ent0.date, 24)

                # Quando existirem entradas extraordinárias de adiantamento no plano, o "DCT" será calculado
                # utilizando as datas do fluxo regular.
                #
                elif ent1.dct_override:
                    dct = (ent1.dct_override.date_to - ent1.dct_override.date_from).days

                    if ent1.dct_override.predates_first_amortization:
                        dct = _diff_surrounding_dates(ent1.dct_override.date_from, 24)

                if ent0.dct_override:
                    dct = (ent1.date - ent0.dct_override.date_from).days

                    if ent0.dct_override.predates_first_amortization:
                        dct = _diff_surrounding_dates(ent0.dct_override.date_from, 24)

                f_s = calculate_interest_factor(apy, decimal.Decimal(dcp) / (12 * decimal.Decimal(dct)))

                if type(ent1) is Amortization.Bare or type(ent1) is Amortization and ent1.price_level_adjustment:
                    kw2: t.Dict[str, t.Any] = {}
                    dcp = (due - ent0.date).days  # Spec "30/360" needs a ratio for the IPCA factor.
                    dct = (ent1.date - ent0.date).days

                    if type(ent1) is Amortization and ent1.price_level_adjustment:
                        kw2['base'] = ent1.price_level_adjustment.base_date
                        kw2['period'] = ent1.price_level_adjustment.period
                        kw2['shift'] = ent1.price_level_adjustment.shift
                        kw2['ratio'] = _1  # Ajuste para a última taxa de correção.

                    else:
                        kw2['base'] = amortizations[0].date.replace(day=1)
                        kw2['period'] = _delta_months(ent1.date, amortizations[0].date)
                        kw2['shift'] = 'M-1'  # FIXME.
                        kw2['ratio'] = _1  # Ajuste para a última taxa de correção.

                    # Exclusivamente para a primeira data de aniversário o "DCT" será considerado como a diferença
                    # em dias corridos entre os dias 24 anterior e posterior à data de início do rendimento.
                    #
                    if ent1.dct_override and num == 1:
                        dct = _diff_surrounding_dates(ent0.date, 24)

                    # Quando existirem entradas extraordinárias de adiantamento no plano, o "DCT" será calculado
                    # utilizando as datas do fluxo regular.
                    #
                    elif ent1.dct_override:
                        dct = (ent1.dct_override.date_to - ent1.dct_override.date_from).days

                        if ent1.dct_override.predates_first_amortization:
                            dct = _diff_surrounding_dates(ent1.dct_override.date_from, 24)

                    if ent0.dct_override:
                        dct = (ent1.date - ent0.dct_override.date_from).days

                        if ent0.dct_override.predates_first_amortization:
                            dct = _diff_surrounding_dates(ent0.dct_override.date_from, 24)

                    kw2['ratio'] = decimal.Decimal(dcp) / decimal.Decimal(dct)

                    f_c = max(vir.backend.calculate_ipca_factor(**kw2), _1)  # Trava o fator de correção.

            elif vir:
                raise NotImplementedError(f'Combination of variable interest rate {vir} and capitalisation {capitalisation} unsupported')

            else:
                raise NotImplementedError(f'Unsupported capitalisation {capitalisation} for fixed interest rate')

        # Fase B.1, FRU, ou Fase Rafa Um.
        #
        # Usando os fatores calculados na fase anterior, calcula e registra as variações de principal, juros e correção
        # monetária.
        #
        # [FATOR-AJUSTE]
        #
        # A inserção de uma antecipação parcial na montagem do cronograma faz com que os percentuais de amortização do
        # principal posteriores à essa antecipação precisem ser atualizados. Essa atualização é feita de forma que o
        # novo percentual de amortização (Pn), de uma prestação arbitrária, deve ser igual ao percentual antigo (Pa)
        # multiplicado por um fator de ajuste (ADJ).
        #
        #                                                  ACUR
        #                                          ADJ = ————————
        #                                                  AREG
        #
        # Em que ACUR é o percentual de amortização restante do fluxo de pagamentos, em cima do principal e incluindo
        # as amortizações extraordinárias (antecipações), e AREG é o percentual de amortização restante do fluxo de
        # pagamentos ordinário.
        #
        if ent0.date < calc_date.value or ent1.date <= calc_date.value or calc_date.runaway:
            # Registra o valor do juros incorrido no período.
            gens.interest_tracker_1.send(calc_balance() * (f_s - _1))

            # Registra a correção do período (FIXME: implementar).
            # gens.price_level_tracker_1.send(…)

            # Caso de uma amortização regular.
            if type(ent1) is Amortization:
                adj = (_1 - regs.principal.amortization_ratio.current) / (_1 - regs.principal.amortization_ratio.regular)  # [FATOR-AJUSTE].

                # Registra o percentual de amortização do principal.
                gens.principal_tracker_1.send(ent1.amortization_ratio * adj)

                # Registra o percentual regular de amortização do principal.
                gens.principal_tracker_2.send(ent1.amortization_ratio)

                # Registra o valor de juros a pagar no período.
                if ent1.amortizes_interest:
                    gens.interest_tracker_2.send(regs.interest.current + regs.principal.amortization_ratio.current * regs.interest.deferred)

                # Registra o valor de correção a pagar no período (FIXME: implementar).
                # gens.price_level_tracker_2.send(…)

            # Caso de um adiantamento (amortização extraordinária).
            #
            # Lembre-se que um adiantamento apresenta apenas um valor bruto que será pago em uma determinada data. Esse
            # valor bruto será fatorado em diversos componentes da dívida, de forma ordenada. O primeiro componente da
            # dívida a ser amortizado é o juro (spread). Após o pagamento dos juros, o que sobra deve ser deduzido da
            # correção monetária. Finalmente, abate-se o valor restante do principal. No bloco de código abaixo,
            #
            #  • "val1" é o valor de juros a pagar.
            #
            #  • "val2" é o valor da correção a pagar. FIXME: a variável "plfv" deveria ser multiplicada pelo
            #    percentual de amortização do principal do período, e não pelo decimal um.
            #
            #  • "val3" é o valor a amortizar do principal.
            #
            # Observe que a ordem de cálculo dessas variáveis corresponde à ordem de fatoração do valor bruto da
            # antecipação.
            #
            else:
                ent1 = t.cast(Amortization.Bare, ent1)  # O Mypy não consegue inferir o tipo da variável "ent1" aqui.
                plfv = principal * (_1 - regs.principal.amortization_ratio.current) * (f_c - _1)  # Price level, full value.
                val0 = min(ent1.value, calc_balance())
                val1 = min(val0, regs.interest.accrued - regs.interest.settled.total)
                val2 = min(val0 - val1, plfv * _1)
                val3 = val0 - val1 - val2

                # Verifica se o valor do pagamento irregular não ultrapassa o saldo em aberto.
                if ent1.value != Amortization.Bare.MAX_VALUE and ent1.value > _Q(calc_balance()):
                    raise Exception(f'the value of the amortization, {ent1.value}, is greater than the remaining balance of the loan, {_Q(calc_balance())}')

                # Registra o percentual de amortização do principal.
                gens.principal_tracker_1.send(val3 / principal)

                # Registra o valor de juros a pagar no período.
                gens.interest_tracker_2.send(val1)

                # Registra o valor de correção a pagar no período (FIXME: implementar).
                # gens.price_level_tracker_2.send(val2)

        # Fase B.2, FRD, ou Fase Rafa Dois.
        #
        # Monta a instância do pagamento, saída da rotina. Faz arredondamentos.
        #
        if ent0.date < calc_date.value or ent1.date <= calc_date.value or calc_date.runaway:
            pmt = PriceAdjustedPayment() if vir and vir.code == 'IPCA' else Payment()

            # B.2.1. Monta o pagamento (PMT).
            pmt.no = num
            pmt.date = ent1.date

            if type(ent1) is Amortization:
                pmt.amort = regs.principal.amortized.current

                if gain_output == 'deferred':
                    pmt.gain = regs.interest.deferred + regs.interest.current

                elif gain_output == 'settled':
                    pmt.gain = regs.interest.settled.current if ent1.amortizes_interest else _0

                else:  # Implies "gain_output == 'current'."
                    pmt.gain = regs.interest.current

                pmt.bal = calc_balance()

                # Amortiza principal, não incorpora juros.
                if pmt.amort and ent1.amortizes_interest:
                    pmt.raw = pmt.amort + (j_f := regs.interest.settled.current if ent1.amortizes_interest else _0)
                    pmt.tax = j_f * calculate_revenue_tax(amortizations[0].date, due)

                # Amortiza principal, incorpora juros.
                elif pmt.amort:
                    pmt.raw = pmt.amort
                    pmt.tax = _0

                # Não amortiza principal, não incorpora juros.
                elif ent1.amortizes_interest:
                    pmt.raw = j_f = regs.interest.settled.current if ent1.amortizes_interest else _0
                    pmt.tax = j_f * calculate_revenue_tax(amortizations[0].date, due)

                # Não amortiza principal, incorpora juros.
                else:
                    pmt.raw = _0
                    pmt.tax = _0

            else:
                pmt.amort = regs.principal.amortized.current

                if gain_output == 'deferred':
                    pmt.gain = regs.interest.deferred + regs.interest.current

                elif gain_output == 'settled':
                    pmt.gain = regs.interest.settled.current

                else:  # Implies "gain_output == 'current'."
                    pmt.gain = regs.interest.current

                pmt.bal = calc_balance()
                pmt.raw = pmt.amort + (j_f := regs.interest.settled.current)
                pmt.tax = j_f * calculate_revenue_tax(amortizations[0].date, due)

            # Aplica a correção no bruto e no I.R.
            if vir and vir.code == 'IPCA':
                pmt = t.cast(PriceAdjustedPayment, pmt)

                pmt.pla = regs.principal.amortized.current * (f_c - _1)
                pmt.raw = pmt.raw + pmt.pla
                pmt.tax = pmt.tax + pmt.pla * calculate_revenue_tax(amortizations[0].date, due)

            # Sanity check.
            #
            # Esse teste de sanidade só é necessário caso três critérios sejam atendidos:
            #
            #   1. Que a entrada do cronograma seja uma antecipação, "Amortization.Bare".
            #
            #   2. Que a antecipação esteja na data de cálculo. Se não estiver, obviamente, os valores não vão bater.
            #
            #   3. Que o valor da antecipação não seja "Amortization.Bare.MAX_VALUE". Nesse caso a rotina usaria o
            #   saldo devedor na data do cálculo como valor da antecipação. Não haveria "input" a ser validado.
            #
            if type(ent1) is Amortization.Bare and ent1.date == calc_date.value and ent1.value < Amortization.Bare.MAX_VALUE:
                assert _Q(pmt.raw) == _Q(ent1.value)

            # B.2.2. Arredonda valores do pagamento, e calcula o seu valor líquido.
            if tax_exempt:
                pmt.tax = _0

            pmt.amort = _Q(pmt.amort)
            pmt.gain = _Q(pmt.gain)
            pmt.raw = _Q(pmt.raw)
            pmt.tax = _Q(pmt.tax)
            pmt.net = pmt.raw - pmt.tax
            pmt.bal = _Q(pmt.bal)

            if vir and vir.code == 'IPCA':
                pmt = t.cast(PriceAdjustedPayment, pmt)

                pmt.pla = _Q(pmt.pla)

            yield pmt

            if pmt.bal == _0:
                break  # Se o saldo é zero, o cronograma acabou.
# }}}

# Daily returns. {{{
@typeguard.typechecked
def get_daily_returns(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    amortizations: t.List[Amortization | Amortization.Bare], *,
    vir: t.Optional[VariableIndex] = None,
    capitalisation: _CAPITALISATION = '360'
) -> t.Iterable[DailyReturn]:
    '''
    Gera uma tabela de rendimentos para um determinado empréstimo.

    Esta função tem assinatura semelhante a "fincore.get_payments_table". Para saber como invocar essa função, tome
    como base a frase abaixo.

      “ Retorne os rendimentos diários de um empréstimo em um valor V, a uma taxa anual TA, nas datas D. ”

    Dessa elaboração saem os três parâmetros obrigatórios e posicionais dessa rotina:

      • "principal", é o valor principal do empréstimo, ou V.

      • "apy", é a taxa nominal anual de spread TA (annual percentage yield).

      • "amortizations", que é uma lista contendo as datas D em que amortizações devem ser realizadas.

    Dois outros parâmetros são opcionais "vir", que especifica um índice variável, podendo ser CDI, Poupança, IPCA, ou
    IGPM; e "capitalisation", que configura a forma de composição dos juros. Veja a documentação da rotina
    "fincore.get_payments_table" para mais detalhes sobre esses parâmetros.

    Emite uma lista de objetos "DailyReturn", que contêm as informações diárias da posição do empréstimo:

      • "date", é a data do rendimento.

      • "value", é o valor rendimento do dia.

      • "bal", é o saldo devedor do empréstimo no final do dia, isto é, considerando o rendimento do dia, e pagamentos
        extraordinários porventura realizados. O saldo inicial do dia D, "D.bal", deve ser obviamente, igual ao valor
        do saldo o final do dia anterior, "D₋₁.bal".

        Fique atento pois a expressão "D.bal - D.value" não dá o valor do saldo inicial do dia. Por dois motivos:

        1. Erros de arredondamento. A saída dessa rotina é quantizada, mas sua memória interna não é. Espere diferença
           de um centavo esporadicamente.

        2. Vão ter dias em que pagamentos serão realizados, e essa rotina não os retorna. De fato, a fórmula do saldo
           inicial é "D.bal - Σ D.entradas + Σ D.saidas", sendo que "D.entradas" seriam as entradas do dia, ou seja, o
           rendimento; e "D.saidas" as saídas, ou pagamentos, do dia. Esse cálculo também padece de erros de
           arredondamentos, pelo mesmo motivo do item anterior: a rotina quantiza valores internos somente antes de
           retorná-los.

      • "fixed_factor", é o fator de juros usado para calcular o componente fixo do rendimento do dia.

      • "variable_factor", é o fator de juros usado para calcular o componente variável do rendimento do dia.
    '''

    # Some indexes are only published by supervisor bodies on business days. For example, Brazilian DI. On such cases
    # this function will fill in the gaps, i.e., provide a zero value if the upstream misses it.
    #
    def get_normalized_cdi_indexes(backend: IndexStorageBackend) -> t.Iterator[decimal.Decimal]:
        for i, (daily0, daily1) in enumerate(itertools.pairwise(backend.get_cdi_indexes(amortizations[0].date, amortizations[-1].date))):
            # Preenche lacunas anteriores ao primeiro índice.
            if i == 0:
                d0 = amortizations[0].date

                while d0 < daily0.date:
                    yield _0

                    d0 += datetime.timedelta(days=1)

            # Entrega o índice corrente.
            yield daily0.value / decimal.Decimal(100)

            # Preenche lacunas.
            delta = (daily1.date - daily0.date).days

            while delta > 1:
                yield _0

                delta -= 1

        # Preenche lacunas após o último índice.
        d1 = daily1.date

        while d1 <= amortizations[-1].date:
            yield _0

            d1 += datetime.timedelta(days=1)

    # Poupança is a monthly index. This function will normalize it to daily values.
    def get_normalized_savings_indexes(backend: IndexStorageBackend) -> t.Iterator[decimal.Decimal]:
        for ranged in backend.get_savings_indexes(amortizations[0].date, amortizations[-1].date):
            init = max(amortizations[0].date, ranged.begin_date)
            ends = min(amortizations[-1].date, ranged.end_date)
            days = (ranged.end_date - ranged.begin_date).days
            rate = calculate_interest_factor(ranged.value, _1 / decimal.Decimal(days))

            for d in _date_range(init, ends):
                yield rate - _1

    # IPCA is a monthly index. This function will normalize it to daily values.
    def get_normalized_ipca_indexes(backend: IndexStorageBackend) -> t.Iterator[decimal.Decimal]:
        raise NotImplementedError()

    def calc_balance() -> decimal.Decimal:
        val = principal * f_c + regs.interest.accrued - regs.principal.amortized.total * f_c - regs.interest.settled.total

        return t.cast(decimal.Decimal, val)

    # Primeiro gerador para valores de principal.
    #
    #  • "principal.amortization_ratio.current", é o percentual de amortização do período corrente.
    #  • "principal.amortized.current", é o valor amortizado no período corrente.
    #  • "principal.amortized.total", é o valor amortizado total (período corrente somado aos passados).
    #
    def track_principal_1() -> t.Generator[None, decimal.Decimal | None, None]:
        while True:
            ratio = yield

            # Se o percentual de amortização atual somado ao acumulado ultrapassar 100%, um reajuste deve ser feito.
            if regs.principal.amortization_ratio.current + ratio > _1:
                ratio = _1 - regs.principal.amortization_ratio.current

            if ratio:
                regs.principal.amortization_ratio.current += ratio
                regs.principal.amortized = types.SimpleNamespace(current=ratio * principal, total=regs.principal.amortization_ratio.current * principal)

            else:
                regs.principal.amortized = types.SimpleNamespace(current=_0, total=regs.principal.amortized.total)

    # Segundo gerador para valores de principal.
    #
    #  • "principal.amortization_ratio.regular", é o percentual de amortização regular acumulado (período corrente somado a passados)
    #
    def track_principal_2() -> t.Generator[None, decimal.Decimal | None, None]:
        while True:
            ratio = yield

            # Se o percentual de amortização regular somado ao acumulado ultrapassar 100%, um reajuste deve ser feito.
            if regs.principal.amortization_ratio.regular + ratio > _1:
                ratio = _1 - regs.principal.amortization_ratio.regular

            if ratio:
                regs.principal.amortization_ratio.regular += ratio

    # Gerador para valores de juros.
    #
    #   • "interest.daily" são os juros incorridos (produzidos) no dia.
    #   • "interest.current" são os juros incorridos (produzidos) no período corrente.
    #   • "interest.accrued" é o total de juros acumulado desde o dia zero do cronograma de pagamentos.
    #   • "interest.deferred" é o total de juros em aberto de períodos passados.
    #
    def track_interest_1() -> t.Generator[None, decimal.Decimal | None, None]:
        while True:
            regs.interest.daily = yield
            regs.interest.current += regs.interest.daily
            regs.interest.accrued += regs.interest.daily
            regs.interest.deferred = regs.interest.accrued - (regs.interest.current + regs.interest.settled.total)

    # Gerador para valores de juros acertados entre devedor e credor.
    #
    #   • "interest.settled.current" são os juros acertados no período corrente.
    #   • "interest.settled.total" é o total de juros acertados desde o dia zero do cronograma de pagamentos.
    #
    def track_interest_2() -> t.Generator[None, decimal.Decimal | None, None]:
        while True:
            regs.interest.settled = types.SimpleNamespace(current=(yield), total=regs.interest.settled.total)
            regs.interest.settled.total += regs.interest.settled.current
            regs.interest.current -= regs.interest.settled.current

    # A. Valida e prepara para execução.
    gens = types.SimpleNamespace()
    regs = types.SimpleNamespace()
    aux = _0

    if principal == _0:
        return

    if principal < _CENTI:
        raise ValueError('principal value should be at least 0.01')

    if len(amortizations) < 2:
        raise ValueError('at least two amortizations are required: the start of the schedule, and its end')

    elif not vir and capitalisation == '252':
        raise ValueError('fixed interest rates should not use the 252 working days capitalisation')

    elif vir and vir.code == 'CDI' and capitalisation != '252':
        raise ValueError('CDI should use the 252 working days capitalisation')

    for i, x in enumerate(amortizations):
        if type(x) is Amortization:
            aux += x.amortization_ratio

        if vir and vir.code not in t.get_args(_PL_INDEX) and type(x) is Amortization and x.price_level_adjustment:
            raise TypeError(f"amortization {i} has price level adjustment, but a variable index wasn't provided, or isn't IPCA nor IGPM")

        elif aux > _1 and not math.isclose(aux, _1):
            raise ValueError('the accumulated percentage of the amortizations overflows 1.0')

    if not math.isclose(aux, _1):
        raise ValueError('the accumulated percentage of the amortizations does not reach 1.0')

    # Registradores.
    regs.interest = types.SimpleNamespace(current=_0, accrued=_0, settled=types.SimpleNamespace(current=_0, total=_0), deferred=_0)
    regs.principal = types.SimpleNamespace(amortization_ratio=types.SimpleNamespace(current=_0, regular=_0), amortized=types.SimpleNamespace(current=_0, total=_0))
    regs.correction = types.SimpleNamespace(current=_1, accrued=_1)

    # Control, create generators.
    gens.interest_tracker_1 = track_interest_1()
    gens.interest_tracker_2 = track_interest_2()
    gens.principal_tracker_1 = track_principal_1()
    gens.principal_tracker_2 = track_principal_2()

    # Control, start generators.
    gens.principal_tracker_1.send(None)
    gens.principal_tracker_2.send(None)
    gens.interest_tracker_1.send(None)
    gens.interest_tracker_2.send(None)

    # List of indexes.
    if vir and vir.code == 'CDI':
        idxs = get_normalized_cdi_indexes(vir.backend)

    elif vir and vir.code == 'Poupança':
        idxs = get_normalized_savings_indexes(vir.backend)

    elif vir:  # Implies "vir.code == 'IPCA'".
        idxs = get_normalized_ipca_indexes(vir.backend)

    # B. Executa.
    itr = iter(amortizations)
    tup = next(itr), next(itr)
    cnt = p = 1

    for ref in _date_range(amortizations[0].date, amortizations[-1].date):
        f_c = _1  # Taxa (ou fator) de correção, FC.
        f_v = _1  # Taxa (ou fator) variável, FV.
        f_s = _1  # Taxa (ou fator) fixo, FS.

        # Fase B.0, FZA, ou Fase Zille-Anna.
        #
        #  • Calcula FS (fator de spread) para índice pré-fixado; e ambos FS e FC para índice de correção.
        #  • Calcula FS para índice pós-fixado (CDI, Poupança etc). Nesse caso não há correção.
        #
        # Altamente alterada com relação à FZA da rotina "get_payments_table".
        #
        if not vir and capitalisation == '360':  # Bullet.
            f_s = calculate_interest_factor(apy, _1 / decimal.Decimal(360))

        elif not vir and capitalisation == '365':  # Bullet in legacy mode.
            f_s = calculate_interest_factor(apy, _1 / decimal.Decimal(365))

        elif not vir and capitalisation == '30/360':  # Juros mensais, Price, Livre.
            v01 = calculate_interest_factor(apy, _1 / decimal.Decimal(12)) - _1  # Fator mensal.

            # O período um tem tratamento especial aqui, para lidar com variações no aniversário do empréstimo.
            #
            # Exemplo, o projeto da Resolvvi de junho de 2023. O primeiro período tem 32 dias, em vez dos esperados 30
            # dias – de 19/06/2023, inclusive, a 21/07/2023, exclusive. A data final do período um foi deslocada em
            # decorrência da data de aniversário do projeto, que é 21/12/2025.
            #
            # Para contemplar os casos em que a data final do primeiro período do empréstimo foi adulterada em relação
            # ao seu início de rendimento, devido a alterações no aniversário do empréstimo, o cálculo do fator de
            # juro fixo emprega a diferença de dias entre as datas inicial e final do período.
            #
            # Em um outro período qualquer, basta saber a quantidade de dias que tem o mês em que ele se inicia.
            #
            # Observe que pagamentos extraordinários não definem os intervalos de um período de um cronograma de
            # amortizações. Por isso testa-se pelo tipo de "tup[1]" abaixo.
            #
            if p == 1 and (type(tup[1]) is Amortization.Bare or ref < tup[1].date):
                v02 = decimal.Decimal((amortizations[1].date - amortizations[0].date).days)  # Dias no período.

            elif ref == tup[1].date:
                v02 = decimal.Decimal(calendar.monthrange(tup[1].date.year, tup[1].date.month)[1])  # Dias do mês do período.

            else:
                v02 = decimal.Decimal(calendar.monthrange(tup[0].date.year, tup[0].date.month)[1])  # Dias do mês do período.

            f_s = calculate_interest_factor(v01, _1 / v02, False)  # Fator diário.

        elif vir and vir.code == 'CDI' and capitalisation == '252':  # Bullet, Juros mensais, Livre.
            f_v = next(idxs) * vir.percentage / decimal.Decimal(100) + _1

            # Lembrar que índice na base 252 só rende em dia útil. Assim funciona o CDI. Nesse caso o fator fixo
            # deve acompanhar o variável. Só deve ser calculado em dia útil.
            #
            # FIXME: se porventura o índice variável, "next(idxs)", for zero, e se tratar de um dia útil, o fator
            # fixo não será calculado. Nunca vi o CDI ser zero, mas vale contemplar esse caso. O correto abaixo é
            # testar se o dia é útil, e não se o valor do fator "f_c" é maior que um.
            #
            if f_v > _1:
                f_s = calculate_interest_factor(apy, _1 / decimal.Decimal(252))

        elif vir and vir.code == 'Poupança' and capitalisation == '360':  # Poupança só suportada em Bullet.
            f_s = calculate_interest_factor(apy, _1 / decimal.Decimal(360))
            f_v = next(idxs) * vir.percentage / decimal.Decimal(100) + _1

        elif vir and vir.code == 'IPCA' and capitalisation == '360':  # Bullet.
            raise NotImplementedError()  # FIXME: implementar.

        elif vir and vir.code == 'IPCA' and capitalisation == '30/360':  # Juros mensais e Livre.
            raise NotImplementedError()  # FIXME: implementar.

        elif vir:
            raise NotImplementedError(f'Combination of variable interest rate {vir} and capitalisation {capitalisation} unsupported')

        else:
            raise NotImplementedError(f'Unsupported capitalisation {capitalisation} for fixed interest rate')

        # Fase B.1, FRU, ou Fase Rafa Um. Levemente alterada com relação à FRU da rotina "get_payments_table".
        while ref == tup[1].date:
            if type(tup[1]) is Amortization:  # Caso de uma amortização regular.
                adj = (_1 - regs.principal.amortization_ratio.current) / (_1 - regs.principal.amortization_ratio.regular)  # [FATOR-AJUSTE].

                # Registra o percentual de amortização do principal.
                gens.principal_tracker_1.send(tup[1].amortization_ratio * adj)

                # Registra o percentual regular de amortização do principal.
                gens.principal_tracker_2.send(tup[1].amortization_ratio)

                # Registra o valor de juros a pagar no período.
                if tup[1].amortizes_interest:
                    gens.interest_tracker_2.send(regs.interest.current + regs.principal.amortization_ratio.current * regs.interest.deferred)

                # Registra o valor de correção a pagar no período (FIXME: implementar).
                # gens.price_level_tracker_2.send(…)

                # Encerra o acumulador de juros do período anterior.
                regs.interest.current = _0

                p += 1  # O período só incrementa em amortizações regulares.

                cnt = 1

            # Caso de um adiantamento (amortização extraordinária).
            #
            # Lembre-se que um adiantamento apresenta apenas um valor bruto que será pago em uma determinada data. Esse
            # valor bruto será fatorado em diversos componentes da dívida, de forma ordenada. O primeiro componente da
            # dívida a ser amortizado é o juro (spread). Após o pagamento dos juros, o que sobra deve ser deduzido da
            # correção monetária. Finalmente, abate-se o valor restante do principal. No bloco de código abaixo,
            #
            #  • "val1" é o valor de juros a pagar.
            #
            #  • "val2" é o valor da correção a pagar. FIXME: a variável "plfv" deveria ser multiplicada pelo
            #    percentual de amortização do principal do período, e não pelo decimal um.
            #
            #  • "val3" é o valor a amortizar do principal.
            #
            # Observe que a ordem de cálculo dessas variáveis corresponde à ordem de fatoração do valor bruto da
            # antecipação.
            #
            else:
                ent = t.cast(Amortization.Bare, tup[1])  # O Mypy não consegue inferir o tipo da variável "ent" aqui.
                plfv = principal * (_1 - regs.principal.amortization_ratio.current) * (f_c - _1)  # Price level, full value.
                val0 = min(ent.value, calc_balance())
                val1 = min(val0, regs.interest.accrued - regs.interest.settled.total)
                val2 = min(val0 - val1, plfv * _1)
                val3 = val0 - val1 - val2

                # Verifica se o valor do pagamento irregular não ultrapassa o saldo em aberto.
                if ent.value != Amortization.Bare.MAX_VALUE and ent.value > _Q(calc_balance()):
                    raise Exception(f'the value of the amortization, {ent.value}, is greater than the remaining balance of the loan, {_Q(calc_balance())}')

                # Registra o percentual de amortização do principal.
                gens.principal_tracker_1.send(val3 / principal)

                # Registra o valor de juros a pagar no período.
                gens.interest_tracker_2.send(val1)

                # Registra o valor de correção a pagar no período (FIXME: implementar).
                # gens.price_level_tracker_2.send(val2)

                # Encerra o acumulador de juros do período anterior.
                regs.interest.current = _0

            tup = tup[1], next(itr)

        # Registra o valor do juros incorrido no dia.
        gens.interest_tracker_1.send(calc_balance() * (f_s * f_v * f_c - _1))

        # Registra a correção do período (FIXME: implementar).
        # gens.price_level_tracker_1.send(…)

        # Se o saldo é zero, o cronograma acabou.
        if _Q(calc_balance()) == _0:
            break

        # Monta a instância de rendimento diário, saída da rotina. Faz arredondamentos.
        dr = DailyReturn()

        dr.no = cnt
        dr.period = p
        dr.date = ref
        dr.value = _Q(regs.interest.daily)
        dr.bal = _Q(calc_balance())

        dr.fixed_factor = f_s
        dr.variable_factor = f_v * f_c

        yield dr

        cnt += 1
# }}}

# Factories. {{{
def preprocess_bullet(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    zero_date: datetime.date,
    term: int,
    insertions: t.List[Amortization.Bare] = [],
    anniversary_date: t.Optional[datetime.date] = None,
    capitalisation: _DAILY_CAPITALISATION = '360',
    vir: t.Optional[VariableIndex] = None,
    calc_date: t.Optional[CalcDate] = None  # Pass through.
) -> t.List[Amortization | Amortization.Bare]:
    sched: t.List[Amortization | Amortization.Bare] = []

    # 1. Valida.
    if term <= 0:  # See [ANNOTATED_TYPES] above.
        raise ValueError('"term" must be a greater than, or equal to, one')

    if anniversary_date and anniversary_date <= zero_date:
        raise ValueError(f'the "anniversary_date", {anniversary_date}, must be greater than "zero_date", {zero_date}')

    elif anniversary_date and abs((anniversary_date - (zero_date + _MONTH * term)).days) > 20:
        raise ValueError(f'the "anniversary_date", {anniversary_date}, is more than 20 days away from the regular payment date, {zero_date + _MONTH * term}')

    for i, x in enumerate(insertions):
        if x.value <= 0:
            raise ValueError(f'invalid value for insertion entry #{i} – should be positive')

        elif x.date <= zero_date:
            raise ValueError(f'"insertions[{i}].date", {x.date}, must succeed "zero_date", {zero_date}')

        elif not anniversary_date and x.date > (due := zero_date + _MONTH * term):
            raise ValueError(f'"insertions[{i}].date", {x.date}, succeeds the regular payment date, {due}')

        elif anniversary_date and x.date > anniversary_date:
            raise ValueError(f'"insertions[{i}].date", {x.date}, succeeds "anniversary_date", {anniversary_date}')

    # Base de cálculo 365 somente para pré-fixadas históricas. Recomenda-se o uso da base 360 para pré-fixadas.
    if capitalisation == '365':
        _LOG.warning('capitalising 365 days per year exists solely for legacy Bullet support – prefer 360 days')

    # 2.1. Cria as amortizações. Fluxo regular, sem inserções. Rápido.
    if not insertions and not vir:
        sched.append(Amortization(date=zero_date, amortizes_interest=False))
        sched.append(Amortization(date=anniversary_date or zero_date + _MONTH * term, amortization_ratio=_1))

        if anniversary_date:
            sched[-1].dct_override = DctOverride(anniversary_date, anniversary_date, predates_first_amortization=False)

    # 2.2. Cria as amortizações. Fluxo regular com Índice Nacional de Preços ao Consumidor Amplo, sem inserções. Rápido.
    elif not insertions and vir and vir.code == 'IPCA':
        dif = min(_delta_months(calc_date.value, zero_date), term) if calc_date else term
        pla = PriceLevelAdjustment('IPCA', base_date=zero_date.replace(day=1), period=dif)

        sched.append(Amortization(date=zero_date, amortizes_interest=False))
        sched.append(Amortization(date=anniversary_date or zero_date + _MONTH * term, amortization_ratio=_1, price_level_adjustment=pla))

        if anniversary_date:
            sched[-1].dct_override = DctOverride(anniversary_date, anniversary_date, predates_first_amortization=False)

    # 2.3. Cria as amortizações. Faz inserções no fluxo regular. Lento.
    else:
        lst = []

        lst.append(Amortization(date=zero_date, amortizes_interest=False))
        lst.append(Amortization(date=anniversary_date or zero_date + _MONTH * term, amortization_ratio=_1))

        for skel in _interleave(lst, insertions, key=lambda x: x.date):
            sched.append(skel.item)

            if skel.from_a and vir and vir.code == 'IPCA':
                dif = _delta_months(skel.item.date, zero_date)

                skel.item.price_level_adjustment = PriceLevelAdjustment('IPCA')

                skel.item.price_level_adjustment.base_date = zero_date.replace(day=1)
                skel.item.price_level_adjustment.period = dif
                skel.item.price_level_adjustment.amortizes_adjustment = skel.index_a == len(lst) - 1

            elif skel.from_b and anniversary_date:
                skel.item.dct_override = DctOverride(zero_date, anniversary_date, predates_first_amortization=True)

            elif skel.from_b:
                skel.item.dct_override = DctOverride(zero_date, zero_date + _MONTH * term, predates_first_amortization=True)

    return sched

def preprocess_jm(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    zero_date: datetime.date,
    term: int,
    insertions: t.List[Amortization.Bare] = [],
    anniversary_date: t.Optional[datetime.date] = None,
    vir: t.Optional[VariableIndex] = None
) -> t.List[Amortization] | t.List[Amortization | Amortization.Bare]:
    lst1 = []
    lst2 = []

    # 1. Valida.
    if term <= 0:  # See [ANNOTATED_TYPES] above.
        raise ValueError('"term" must be a greater than, or equal to, one')

    if anniversary_date and anniversary_date <= zero_date:
        raise ValueError(f'the "anniversary_date", {anniversary_date}, must be greater than "zero_date", {zero_date}')

    elif anniversary_date and abs((anniversary_date - (zero_date + _MONTH)).days) > 20:
        raise ValueError(f'the "anniversary_date", {anniversary_date}, is more than 20 days away from the regular payment date, {zero_date + _MONTH}')

    if vir and vir.code == 'Poupança':
        raise NotImplementedError('"Poupança" is currently unsupported')

    for i, x in enumerate(insertions):
        if x.date <= zero_date:
            raise ValueError(f'"insertions[{i}].date", {x.date}, must succeed "zero_date", {zero_date}')

        elif not anniversary_date and x.date < (due := zero_date + _MONTH):
            raise ValueError(f'"insertions[{i}].date", {x.date}, precedes the fisrt regular payment date, {due}')

        elif anniversary_date and x.date < anniversary_date:
            raise ValueError(f'"insertions[{i}].date", {x.date}, precedes "anniversary_date", {anniversary_date}')

        elif not anniversary_date and x.date > (due := zero_date + _MONTH * term):
            raise ValueError(f'"insertions[{i}].date", {x.date}, succeeds the last regular payment date, {due}')

        elif anniversary_date and x.date > (due := anniversary_date + _MONTH * (term - 1)):
            raise ValueError(f'"insertions[{i}].date", {x.date}, succeeds the last regular payment date, {due}')

    # 2. Cria as amortizações.
    if anniversary_date and anniversary_date == zero_date + _MONTH:
        anniversary_date = None

    # Fluxo regular, sem inserções. Rápido.
    lst1.append(Amortization(date=zero_date, amortizes_interest=False))  # Data zero (início do rendimento).

    for i in range(1, term + 1):
        due = anniversary_date + _MONTH * (i - 1) if anniversary_date else zero_date + _MONTH * i
        ent = Amortization(date=due, amortization_ratio=_0 if i != term else _1)

        if i == 1 and anniversary_date:
            ent.dct_override = DctOverride(anniversary_date, anniversary_date, predates_first_amortization=False)

        if vir and vir.code == 'IPCA':
            ent.price_level_adjustment = PriceLevelAdjustment('IPCA')

            ent.price_level_adjustment.base_date = zero_date.replace(day=1)
            ent.price_level_adjustment.period = i
            ent.price_level_adjustment.amortizes_adjustment = i == term

        lst1.append(ent)

    # Faz inserções no fluxo regular. Lento.
    if insertions:
        for skel in _interleave(lst1, insertions, key=lambda x: x.date):
            lst2.append(skel.item)

            if skel.from_a and vir and vir.code == 'IPCA':
                skel.item.price_level_adjustment = PriceLevelAdjustment('IPCA')

                skel.item.price_level_adjustment.base_date = zero_date.replace(day=1)
                skel.item.price_level_adjustment.period = skel.index_a
                skel.item.price_level_adjustment.amortizes_adjustment = skel.index_a == len(lst1) - 1

            if skel.from_b:
                date1 = lst1[skel.index_a - 1].date
                date2 = lst1[skel.index_a].date

                skel.item.dct_override = DctOverride(date1, date2, predates_first_amortization=skel.index_a == 1)

        return lst2

    return lst1

def preprocess_price(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    zero_date: datetime.date,
    term: int,
    insertions: t.List[Amortization.Bare] = [],
    anniversary_date: t.Optional[datetime.date] = None
) -> t.List[Amortization] | t.List[Amortization | Amortization.Bare]:
    lst1 = []
    lst2 = []

    # 1. Valida.
    if term <= 0:  # See [ANNOTATED_TYPES] above.
        raise ValueError('"term" must be a greater than, or equal to, one')

    if anniversary_date and anniversary_date <= zero_date:
        raise ValueError(f'the "anniversary_date", {anniversary_date}, must be greater than "zero_date", {zero_date}')

    elif anniversary_date and abs((anniversary_date - (zero_date + _MONTH)).days) > 20:
        raise ValueError(f'the "anniversary_date", {anniversary_date}, is more than 20 days away from the regular payment date, {zero_date + _MONTH}')

    for i, x in enumerate(insertions):
        if x.date <= zero_date:
            raise ValueError(f'"insertions[{i}].date", {x.date}, must succeed "zero_date", {zero_date}')

        elif not anniversary_date and x.date < (due := zero_date + _MONTH):
            raise ValueError(f'"insertions[{i}].date", {x.date}, precedes the fisrt regular payment date, {due}')

        elif anniversary_date and x.date < anniversary_date:
            raise ValueError(f'"insertions[{i}].date", {x.date}, precedes "anniversary_date", {anniversary_date}')

        elif not anniversary_date and x.date > (due := zero_date + _MONTH * term):
            raise ValueError(f'"insertions[{i}].date", {x.date}, succeeds the last regular payment date, {due}')

        elif anniversary_date and x.date > (due := anniversary_date + _MONTH * (term - 1)):
            raise ValueError(f'"insertions[{i}].date", {x.date}, succeeds the last regular payment date, {due}')

    # 2. Cria as amortizações.
    if anniversary_date and anniversary_date == zero_date + _MONTH:
        anniversary_date = None

    # Fluxo regular, sem inserções. Rápido.
    lst1.append(Amortization(date=zero_date, amortizes_interest=False))  # Data zero (início do rendimento).

    for i, y in enumerate(amortize_fixed(principal, apy, term), 1):
        due = anniversary_date + _MONTH * (i - 1) if anniversary_date else zero_date + _MONTH * i

        lst1.append(Amortization(date=due, amortization_ratio=y))

        if i == 1 and anniversary_date:
            lst1[-1].dct_override = DctOverride(anniversary_date, anniversary_date, predates_first_amortization=False)

    # Faz inserções no fluxo regular. Lento.
    if insertions:
        for skel in _interleave(lst1, insertions, key=lambda x: x.date):
            lst2.append(skel.item)

            if skel.from_b:
                date1 = lst1[skel.index_a - 1].date
                date2 = lst1[skel.index_a].date

                skel.item.dct_override = DctOverride(date1, date2, predates_first_amortization=skel.index_a == 1)

        return lst2

    return lst1

def preprocess_livre(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    amortizations: t.List[Amortization],
    insertions: t.List[Amortization.Bare] = [],
    vir: t.Optional[VariableIndex] = None
) -> t.List[Amortization | Amortization.Bare]:
    sched: t.List[Amortization | Amortization.Bare] = []
    aux = _0

    # 1. Valida.
    if len(amortizations) < 2:
        raise ValueError('at least two amortizations are required: the start of the schedule, and its end')

    if vir and vir.code == 'Poupança':
        raise NotImplementedError('"Poupança" is currently unsupported')

    for i, x in enumerate(amortizations):
        aux += x.amortization_ratio

        if vir and vir.code not in t.get_args(_PL_INDEX) and type(x) is Amortization and x.price_level_adjustment:
            raise TypeError(f"amortization {i} has price level adjustment, but a variable index wasn't provided, or isn't IPCA nor IGPM")

    for i, y in enumerate(insertions):
        if y.value <= 0:
            raise ValueError(f'invalid value for insertion entry #{i} – should be positive')

        elif y.date <= (zero_date := amortizations[0].date):
            raise ValueError(f'"insertions[{i}].date", {y.date}, must succeed "zero_date", {zero_date}')

        elif y.date < (due := amortizations[1].date):
            raise ValueError(f'"insertions[{i}].date", {y.date}, precedes the fisrt regular payment date, {due}')

        elif y.date > (due := amortizations[-1].date):
            raise ValueError(f'"insertions[{i}].date", {y.date}, succeeds the last regular payment date, {due}')

    if abs((amortizations[1].date - (amortizations[0].date + _MONTH)).days) > 20:
        raise ValueError(f'the first payment date, {amortizations[1].date}, is more than 20 days away from the regular payment date, {amortizations[0].date + _MONTH}')

    if len(set([z.date for z in amortizations])) != len(amortizations):
        raise ValueError('amortization dates must be unique.')

    if not math.isclose(aux, _1):
        raise ValueError('the accumulated percentage of the amortizations does not reach 1.0')

    # 2. Cria as amortizações.
    if not insertions:  # Fluxo regular, sem inserções.
        sched.extend(amortizations)

    else:  # Fluxo extraordinário, com inserções.
        for skel in _interleave(amortizations, insertions, key=lambda x: x.date):
            if skel.from_a:
                sched.append(skel.item)

            else:
                amort = Amortization.Bare(date=skel.item.date)
                date1 = amortizations[skel.index_a - 1].date
                date2 = amortizations[skel.index_a].date

                amort.value = skel.item.value
                amort.dct_override = DctOverride(date1, date2, predates_first_amortization=skel.index_a == 1)

                sched.append(amort)

    return sched

# FIXME: remove this class.
class PaymentFactory:
    @staticmethod
    @typeguard.typechecked
    def create(mode: _OP_MODES, **kwa: t.Any) -> t.Iterable[Payment]:
        if mode == 'Bullet':
            return build_bullet(**kwa)

        elif mode == 'Juros mensais':
            return build_jm(**kwa)

        elif mode == 'Price':
            return build_price(**kwa)

        else:
            return build(**kwa)

# FIXME: renomear para "get_bullet_payments".
@typeguard.typechecked
def build_bullet(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    zero_date: datetime.date,
    term: int, *,  # Junto com "zero_date", equivale ao "amortizations".

    # Optional list of non-scheduled amortizations, a.k.a. prepayments.
    insertions: t.List[Amortization.Bare] = [],

    # Etc.
    anniversary_date: t.Optional[datetime.date] = None,
    vir: t.Optional[VariableIndex] = None,
    calc_date: t.Optional[CalcDate] = None,  # Pass through.
    capitalisation: _DAILY_CAPITALISATION = '360',
    tax_exempt: t.Optional[bool] = False,
    gain_output: _GAIN_OUTPUT_MODE = 'current'
) -> t.Iterable[Payment]:
    '''
    Estereotipa uma operação Bullet.

    Além do valor inicial do principal, e da taxa anual percentual, "apy", essa função recebe a data inicial do
    rendimento, "zero_date", e o prazo do mesmo, "term". Com isso ela gera um cronograma de amortizações Bullet, e dele
    deriva os valores dos pagamentos, através da chamada a "fincore.get_payments_table". Observe que o Bullet se
    caracteriza por apenas um pagamento, mas o cronograma regular tem duas entradas: a do início do rendimento e a
    final, de amortização total.

    Além disso, quando deseja-se simular um ou mais pagamentos antecipados, totais ou parciais, essa função executa os
    pré-requisitos.

      • Insere as antecipações no fluxo regular Bullet.

      • Aplica a correção monetária informada no parâmetro "vir". Usa deslocamento "M-1", automaticamente. A
        correção acumula-se até o último pagamento, seja ele planejado, seja uma antecipação total.

      • Usa automaticamente o ajuste "360" para operações pré-fixadas, e "252" para pós-fixadas. Aceita "365" para
        operações legadas. Parâmetro "capitalisation".

    Antecipações vão no parâmetro "insertions". É uma lista de objetos "obj", em que,

      • "obj.date" é a data da antecipação.

      • "obj.amortization_ratio", é o percentual do principal inicial que será amortizado nesse pagamento antecipado.

    Os parâmetros "vir" e "calc_date" são os mesmos de "fincore.get_payments_table".
    '''

    kwa: t.Dict[str, t.Any] = {}

    kwa['principal'] = principal
    kwa['apy'] = apy
    kwa['amortizations'] = preprocess_bullet(principal, apy, zero_date, term, insertions, anniversary_date, capitalisation, vir, calc_date)

    kwa['vir'] = vir
    kwa['capitalisation'] = '252' if vir and vir.code == 'CDI' else capitalisation

    kwa['calc_date'] = calc_date
    kwa['tax_exempt'] = tax_exempt
    kwa['gain_output'] = gain_output

    yield from get_payments_table(**kwa)

# FIXME: renomear para "get_jm_payments".
@typeguard.typechecked
def build_jm(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    zero_date: datetime.date,
    term: int, *,  # Junto com "zero_date", equivale ao "amortizations".

    # Optional list of non-scheduled amortizations, a.k.a. prepayments.
    insertions: t.List[Amortization.Bare] = [],

    # Etc.
    anniversary_date: t.Optional[datetime.date] = None,
    vir: t.Optional[VariableIndex] = None,
    calc_date: t.Optional[CalcDate] = None,  # Pass through.
    tax_exempt: t.Optional[bool] = False,
    gain_output: _GAIN_OUTPUT_MODE = 'current'
) -> t.Iterable[Payment]:
    '''
    Estereotipa uma operação Juros Mensais.

    Além do valor inicial do principal, e da taxa anual percentual, "apy", essa função recebe a data inicial do
    rendimento, "zero_date", e o prazo do mesmo, "term". Com isso ela gera um cronograma de amortizações Price, e dele
    deriva os valores dos pagamentos, através da chamada a "fincore.get_payments_table".

    Além disso, quando deseja-se simular um ou mais pagamentos antecipados, totais ou parciais, essa função executa os
    pré-requisitos.

      • Intercala as antecipações com o fluxo regular dos juros mensais em um cronograma consolidado.

      • Aplica a correção monetária informada no parâmetro "vir". Usa deslocamento "M-1", automaticamente. A
        correção acumula-se até o último pagamento, seja ele planejado, seja uma antecipação total.

      • Usa automaticamente o quociente "30/360" para operações pré-fixadas, e "252" para pós-fixadas.

    Antecipações vão no parâmetro "insertions". É uma lista de objetos "obj", em que,

      • "obj.date" é a data da antecipação.

      • "obj.amortization_ratio", é o percentual do principal inicial que será amortizado nesse pagamento antecipado.

    Os parâmetros "vir" e "calc_date" são os mesmos de "fincore.get_payments_table".
    '''

    # 3. Gera o cronograma de pagamentos.
    kwa: t.Dict[str, t.Any] = {}

    kwa['principal'] = principal
    kwa['apy'] = apy
    kwa['amortizations'] = preprocess_jm(principal, apy, zero_date, term, insertions, anniversary_date, vir)

    kwa['vir'] = vir
    kwa['capitalisation'] = '252' if vir and vir.code == 'CDI' else '30/360'

    kwa['calc_date'] = calc_date
    kwa['tax_exempt'] = tax_exempt
    kwa['gain_output'] = gain_output

    yield from get_payments_table(**kwa)

# FIXME: renomear para "get_price_payments".
@typeguard.typechecked
def build_price(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    zero_date: datetime.date,
    term: int, *,  # Junto com "zero_date", equivale ao "amortizations".

    # Optional list of non-scheduled amortizations, a.k.a. prepayments.
    insertions: t.List[Amortization.Bare] = [],

    # Etc.
    anniversary_date: t.Optional[datetime.date] = None,
    calc_date: t.Optional[CalcDate] = None,  # Pass through.
    tax_exempt: t.Optional[bool] = False,
    gain_output: _GAIN_OUTPUT_MODE = 'current'
) -> t.Iterable[Payment]:
    '''
    Estereotipa uma operação Price.

    Além do valor inicial do principal, e da taxa anual percentual, "apy", essa função recebe a data inicial do
    rendimento, "zero_date", e o prazo do mesmo, "term". Com isso ela gera um cronograma de amortizações Price, e dele
    deriva os valores dos pagamentos, através da chamada a "fincore.get_payments_table".

    Além disso, quando deseja-se simular um ou mais pagamentos antecipados, totais ou parciais, essa função executa os
    pré-requisitos.

      • Intercala as antecipações com o fluxo regular da Price em um cronograma consolidado.

      • Recalcula os percentuais de amortização pós uma antecipação parcial. Garante que as amortizações recalculadas
        continuem convertendo-se em pagamentos com valor constante, característicos da Price.

      • Usa o quociente "30/360".

    Antecipações vão no parâmetro "insertions". É uma lista de objetos "obj", em que,

      • ""obj".date" é a data da antecipação.

      • ""obj".amortization_ratio", é o percentual do principal inicial que será amortizado nesse pagamento antecipado.

    O parâmetro "calc_date" é o mesmo de "fincore.get_payments_table".
    '''

    # 3. Gera o cronograma de pagamentos.
    kwa: t.Dict[str, t.Any] = {}

    kwa['principal'] = principal
    kwa['apy'] = apy
    kwa['amortizations'] = preprocess_price(principal, apy, zero_date, term, insertions, anniversary_date)

    kwa['capitalisation'] = '30/360'

    kwa['calc_date'] = calc_date
    kwa['tax_exempt'] = tax_exempt
    kwa['gain_output'] = gain_output

    yield from get_payments_table(**kwa)

# FIXME: renomear para "get_livre_payments".
@typeguard.typechecked
def build(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    amortizations: t.List[Amortization], *,

    # Optional list of amortizations.
    insertions: t.List[Amortization.Bare] = [],

    # Etc.
    vir: t.Optional[VariableIndex] = None,
    calc_date: t.Optional[CalcDate] = None,  # Pass through.
    tax_exempt: t.Optional[bool] = False,
    gain_output: _GAIN_OUTPUT_MODE = 'current'
) -> t.Iterable[Payment]:
    '''
    Constrói uma operação Livre.

    Além do valor inicial do principal, e da taxa anual percentual, "apy", essa função recebe duas listas de
    amortizações: uma com o fluxo regular, e outra com as antecipações. Tem duas grandes utilidades:

      1. Intercala os fluxos em ordem de data, antes de finalmente invocar "fincore.get_payments_table".

      2. Calcula os percentuais de amortização, quando há amortização extraregular, i.e., uma antecipação.
         A rotina garante que, caso a antecipação não seja total, as amortizações seguintes a ela gerem pagamentos
         com valor proporcional.

    Os parâmetros "vir" e "calc_date" são os mesmos de "fincore.get_payments_table".
    '''

    # 3. Gera o cronograma de amortizações a partir do esqueleto.
    kwa: t.Dict[str, t.Any] = {}

    kwa['principal'] = principal
    kwa['apy'] = apy
    kwa['amortizations'] = preprocess_livre(principal, apy, amortizations, insertions, vir)

    kwa['vir'] = vir
    kwa['capitalisation'] = '252' if vir and vir.code == 'CDI' else '30/360'

    kwa['calc_date'] = calc_date
    kwa['tax_exempt'] = tax_exempt
    kwa['gain_output'] = gain_output

    yield from get_payments_table(**kwa)

@typeguard.typechecked
def get_bullet_daily_returns(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    zero_date: datetime.date,
    term: int, *,  # Junto com "zero_date", equivale ao "amortizations".

    # Optional list of non-scheduled amortizations, a.k.a. prepayments.
    insertions: t.List[Amortization.Bare] = [],

    # Etc.
    anniversary_date: t.Optional[datetime.date] = None,
    vir: t.Optional[VariableIndex] = None,
    capitalisation: _DAILY_CAPITALISATION = '360'
) -> t.Iterable[DailyReturn]:
    kwa: t.Dict[str, t.Any] = {}

    kwa['principal'] = principal
    kwa['apy'] = apy
    kwa['amortizations'] = preprocess_bullet(principal, apy, zero_date, term, insertions, anniversary_date, capitalisation, vir, calc_date=None)
    kwa['vir'] = vir
    kwa['capitalisation'] = '252' if vir and vir.code == 'CDI' else capitalisation

    yield from get_daily_returns(**kwa)

@typeguard.typechecked
def get_jm_daily_returns(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    zero_date: datetime.date,
    term: int, *,  # Junto com "zero_date", equivale ao "amortizations".
    insertions: t.List[Amortization.Bare] = [],
    anniversary_date: t.Optional[datetime.date] = None,
    vir: t.Optional[VariableIndex] = None
) -> t.Iterable[DailyReturn]:
    kwa: t.Dict[str, t.Any] = {}

    kwa['principal'] = principal
    kwa['apy'] = apy
    kwa['amortizations'] = preprocess_jm(principal, apy, zero_date, term, insertions, anniversary_date, vir)
    kwa['vir'] = vir
    kwa['capitalisation'] = '252' if vir and vir.code == 'CDI' else '30/360'

    yield from get_daily_returns(**kwa)

@typeguard.typechecked
def get_price_daily_returns(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    zero_date: datetime.date,
    term: int, *,  # Junto com "zero_date", equivale ao "amortizations".
    insertions: t.List[Amortization.Bare] = [],
    anniversary_date: t.Optional[datetime.date] = None
) -> t.Iterable[DailyReturn]:
    kwa: t.Dict[str, t.Any] = {}

    kwa['principal'] = principal
    kwa['apy'] = apy
    kwa['amortizations'] = preprocess_price(principal, apy, zero_date, term, insertions, anniversary_date)
    kwa['capitalisation'] = '30/360'

    yield from get_daily_returns(**kwa)

@typeguard.typechecked
def get_livre_daily_returns(
    principal: decimal.Decimal,
    apy: decimal.Decimal,
    amortizations: t.List[Amortization], *,
    insertions: t.List[Amortization.Bare] = [],
    vir: t.Optional[VariableIndex] = None
) -> t.Iterable[DailyReturn]:
    kwa: t.Dict[str, t.Any] = {}

    kwa['principal'] = principal
    kwa['apy'] = apy
    kwa['vir'] = vir
    kwa['amortizations'] = preprocess_livre(principal, apy, amortizations, insertions, vir)
    kwa['capitalisation'] = '252' if vir and vir.code == 'CDI' else '30/360'

    yield from get_daily_returns(**kwa)
# }}}

# Public helpers. {{{
@typeguard.typechecked
def calculate_revenue_tax(begin: datetime.date, end: datetime.date) -> decimal.Decimal:
    '''Calcula imposto para renda fixa.'''

    if end > begin:
        dif = (end - begin).days

        for minimum, maximum, rate in _REVENUE_TAX_BRACKETS:
            if minimum < dif <= maximum:
                return rate

    raise ValueError(f'end date, {end}, should be grater than the begin date, {begin}.')

@functools.cache
@typeguard.typechecked
def calculate_interest_factor(rate: decimal.Decimal, period: decimal.Decimal, percent: bool = True) -> decimal.Decimal:
    '''Calcula o fator de juros dada uma taxa anual percentual (APY) e um período.'''

    if percent:
        rate = decimal.Decimal(rate) / decimal.Decimal(100)

    return (1 + rate) ** decimal.Decimal(period)

@typeguard.typechecked
def calculate_iof(begin: datetime.date, term: int) -> decimal.Decimal:
    '''
    Calcula o IOF para uma aplicação de renda fixa.

    Calcula o IOF para uma aplicação de renda fixa, dados sua data inicial e o
    seu prazo em meses.
    '''

    if term >= 12:
        return decimal.Decimal('1.88')

    else:
        data2 = begin + _MONTH * term
        delta = (data2 - begin).days

        return decimal.Decimal('0.38') + decimal.Decimal('0.00411') * delta

@typeguard.typechecked
def amortize_fixed(principal: decimal.Decimal, apy: decimal.Decimal, term: int) -> t.Generator[decimal.Decimal, None, None]:
    '''
    Constrói uma tabela de amortização para uma aplicação de renda fixa.

    Retorna um iterador de percentuais de amortização.
    '''

    if term > 0:
        fac = calculate_interest_factor(apy, _1 / decimal.Decimal(12))
        pmt = (principal * (fac - _1)) / (_1 - pow(fac, -term))
        bal = principal

        while bal > 0:
            amr = pmt - (bal * (fac - _1)) if bal - pmt >= 0 else bal
            bal = bal - amr

            yield amr / principal

@typeguard.typechecked
def get_late_payment(
    in_pmt: t.Union[LatePayment, LatePriceAdjustedPayment],  # Pagamento.

    # Atraso, data do pagamento.
    calc_date: datetime.date,

    # Dados extras do pagamento. FIXME: os campos abaixo poderiam ser parte da classe Payment, como meta dados.
    apy: decimal.Decimal,  # Taxa anual de juros remuneratórios a.a. (spread).
    zero_date: datetime.date,  # Data inicial do cronograma de pagamentos, para cálculo do I.R.
    vir: t.Optional[VariableIndex] = None,  # Índice variável.
    capitalisation: _CAPITALISATION = '360',  # Base de cálculo para a taxa de juros remuneratórios.

    # Atraso, taxas praticadas.
    fee_rate: decimal.Decimal = LatePayment.FEE_RATE,
    fine_rate: decimal.Decimal = LatePayment.FINE_RATE,

    # Dados extras de índices de correção.
    pla_operations: t.List[t.Tuple[datetime.date, bool, PriceLevelAdjustment]] = []
) -> t.Union[LatePayment, LatePriceAdjustedPayment]:
    '''Gera uma saída de pagamento em atraso.'''

    if not vir and capitalisation == '252':
        raise ValueError('fixed interest rates should not use the 252 working days capitalisation')

    elif vir and vir.code == 'CDI' and capitalisation != '252':
        raise ValueError('CDI should use the 252 working days capitalisation')

    elif vir and vir.code == 'IPCA' and (not pla_operations or any(x for x in pla_operations if x[2].code != 'IPCA')):
        raise ValueError('IPCA should have metadata about the price level adjustment')

    f_1 = f_2 = f_3 = _1

    # A. Pré-fixado, com ou sem correção.
    if not vir or vir.code == 'IPCA' or vir.code == 'IGPM':
        dcp = dct = _1

        if capitalisation in ['360', '365']:
            dcp = decimal.Decimal((calc_date - in_pmt.date).days)
            dct = decimal.Decimal(capitalisation)

        elif capitalisation == '30/360':
            dcp = decimal.Decimal((calc_date - in_pmt.date).days)
            dct = decimal.Decimal(12 * _diff_days_to_same_day_on_prev_month(in_pmt.date))

        else:
            raise NotImplementedError()

        # Fator de juros remuneratórios.
        #
        # Calculado em cima da taxa fixa anual da operação (APY), considerando os dias corridos entre a data prevista
        # para o pagamento ordinário e a data do pagamento em atraso.
        #
        f_1 = calculate_interest_factor(apy, dcp / dct)

        # Fator de juros moratórios.
        #
        # Calculado a partir de uma taxa fixa mensal por atraso (FEE), considerando os dias corridos entre a data
        # prevista para o pagamento ordinário e a data do pagamento em atraso. No entanto, de acordo com a fórmula de
        # juros simples e considerando sempre o mês com duração de 30 dias corridos.
        #
        f_2 = _1 + (fee_rate / decimal.Decimal(100) * dcp / decimal.Decimal(30))

        if vir and vir.code == 'IPCA':
            f_c = _1

            # Composição dos elementos do parâmetro "pla_operations":
            #
            # 1. Data de cálculo do fator de correção.
            # 2. Se deve considerar o período anterior ou posterior à data de cálculo.
            # 3. Informações adicionais para o cálculo do fator de correção (PLA).
            #
            for t01 in pla_operations:
                if t01[2].code == 'IPCA':
                    t02 = ((x := t01[0].replace(day=1)), x + _MONTH)  # Armazena as datas do último e do próximo aniversário.
                    dcp = decimal.Decimal((t01[0] - t02[0]).days) if t01[1] else decimal.Decimal((t02[1] - t01[0]).days)
                    dct = decimal.Decimal((t02[1] - t02[0]).days)
                    kwa: t.Dict[str, t.Any] = {}

                    kwa['base'] = t01[2].base_date
                    kwa['period'] = t01[2].period
                    kwa['shift'] = t01[2].shift
                    kwa['ratio'] = dcp / dct

                    f_c = f_c * vir.backend.calculate_ipca_factor(**kwa)

                else:
                    raise NotImplementedError()

    # B. Pós-fixado (Variable Interest Rate). Implica em CDI ou Poupança.
    elif vir.code == 'CDI' and capitalisation == '252':
        # Fator de juros remuneratórios.
        #
        # Calculado em cima da taxa fixa anual da operação (APY) e da taxa variável referente (VIR), considerando
        # os dias úteis entre a data prevista para o pagamento ordinário e a data do pagamento em atraso.
        #
        f_v = vir.backend.calculate_cdi_factor(in_pmt.date, calc_date, vir.percentage)
        f_s = calculate_interest_factor(apy, decimal.Decimal(f_v.amount) / decimal.Decimal(252))
        f_1 = f_v.value * f_s

        # Fator de juros moratórios.
        #
        # Calculado a partir de uma taxa fixa mensal por atraso (FEE), considerando os dias úteis entre a data
        # prevista para o pagamento ordinário e a data do pagamento em atraso. No entanto, de acordo com a fórmula
        # de juros simples e considerando sempre o mês com duração de 21 dias úteis.
        #
        f_2 = _1 + (fee_rate / decimal.Decimal(100) * decimal.Decimal(f_v.amount) / decimal.Decimal(21))

    else:
        raise NotImplementedError()

    # Fator de multa.
    #
    # A multa é fixa (FINE_RATE). Não há variação do fator de acordo com a extensão do período em atraso.
    #
    if in_pmt.date < calc_date:
        f_3 = _1 + (fine_rate / decimal.Decimal(100))

    if not vir or vir.code == 'CDI':
        gain = in_pmt.gain + in_pmt.extra_gain + in_pmt.penalty + in_pmt.fine
        out_pmt_1 = LatePayment()

        v_1 = (in_pmt.raw) * (f_1 - _1)  # Valor de juros remuneratórios.
        v_2 = (in_pmt.raw + v_1) * (f_2 - _1)  # Valor de juros moratórios.
        v_3 = (in_pmt.raw + v_1 + v_2) * (f_3 - _1)  # Valor de multa.

        v_1 = _Q(v_1)
        v_2 = _Q(v_2)
        v_3 = _Q(v_3)

        gain += (v_1 + v_2 + v_3)

        tax = gain * calculate_revenue_tax(zero_date, calc_date)
        raw = in_pmt.amort + gain

        out_pmt_1.no = in_pmt.no
        out_pmt_1.date = calc_date
        out_pmt_1.raw = raw
        out_pmt_1.tax = _Q(tax)
        out_pmt_1.net = out_pmt_1.raw - out_pmt_1.tax
        out_pmt_1.gain = in_pmt.gain
        out_pmt_1.amort = in_pmt.amort
        out_pmt_1.bal = in_pmt.bal

        out_pmt_1.extra_gain = v_1
        out_pmt_1.penalty = v_2
        out_pmt_1.fine = v_3

        return out_pmt_1

    else:  # Implica em IPCA ou IGPM.
        out_pmt_2 = LatePriceAdjustedPayment()

        raw = _Q(in_pmt.raw * f_c)
        gain = _Q(in_pmt.gain * f_c)
        extra_gain = _Q(in_pmt.extra_gain * f_c)
        penalty = _Q(in_pmt.penalty * f_c)
        fine = _Q(in_pmt.fine * f_c)

        if type(in_pmt) is LatePriceAdjustedPayment:
            pla = _Q(in_pmt.pla + (in_pmt.amort + in_pmt.pla) * (f_c - _1))

        v_1 = (raw) * (f_1 - _1)  # Valor de juros remuneratórios.
        v_2 = (raw + v_1) * (f_2 - _1)  # Valor de juros moratórios.
        v_3 = (raw + v_1 + v_2) * (f_3 - _1)  # Valor de multa.

        v_1 = _Q(v_1)
        v_2 = _Q(v_2)
        v_3 = _Q(v_3)

        tax = (gain + extra_gain + penalty + fine + (v_1 + v_2 + v_3) + pla) * calculate_revenue_tax(zero_date, calc_date)
        raw = raw + (v_1 + v_2 + v_3)

        out_pmt_2.no = in_pmt.no
        out_pmt_2.date = calc_date
        out_pmt_2.raw = raw
        out_pmt_2.tax = _Q(tax)
        out_pmt_2.net = out_pmt_2.raw - out_pmt_2.tax
        out_pmt_2.gain = in_pmt.gain
        out_pmt_2.amort = in_pmt.amort
        out_pmt_2.bal = in_pmt.bal

        if type(in_pmt) is LatePriceAdjustedPayment:
            out_pmt_2.pla = pla

        out_pmt_2.extra_gain = extra_gain + v_1 + (gain - in_pmt.gain)
        out_pmt_2.penalty = penalty + v_2
        out_pmt_2.fine = fine + v_3

        return out_pmt_2
# }}}

if __name__ == '__main__':
    import doctest

    doctest.testmod()

# vi:fdm=marker:


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



# Kichen Sink

In [ ]:
import typing
import decimal
import datetime
import textwrap
import dataclasses

@dataclasses.dataclass
class Bullet:
    principal: decimal.Decimal
    apy: decimal.Decimal
    zero_date: datetime.date
    term: int

    anniversary_date: datetime.date | None = None
    vir: VariableIndex | None = None
    insertions: typing.List[Amortization.Bare] = dataclasses.field(default_factory=list)

    def get_payments(self, calc_date: datetime.date | None) -> typing.Generator[Payment, None, None]:
        kwa = {field.name: getattr(self, field.name) for field in dataclasses.fields(self)}

        kwa['calc_date'] = CalcDate(calc_date) if calc_date else None

        yield from PaymentFactory.create('Bullet', **kwa)

@dataclasses.dataclass
class BulletLegacy:
    principal: decimal.Decimal
    apy: decimal.Decimal
    zero_date: datetime.date
    term: int

    insertions: typing.List[Amortization.Bare] = dataclasses.field(default_factory=list)

    def get_payments(self, calc_date: datetime.date | None) -> typing.Generator[Payment, None, None]:
        kwa = {field.name: getattr(self, field.name) for field in dataclasses.fields(self)}

        kwa['calc_date'] = CalcDate(calc_date) if calc_date else None
        kwa['capitalisation'] = '365'

        yield from PaymentFactory.create('Bullet', **kwa)

@dataclasses.dataclass
class JurosMensais:
    principal: decimal.Decimal
    apy: decimal.Decimal
    zero_date: datetime.date
    term: int

    anniversary_date: datetime.date | None = None
    vir: VariableIndex | None = None
    insertions: typing.List[Amortization.Bare] = dataclasses.field(default_factory=list)

    def get_payments(self, calc_date: datetime.date | None) -> typing.Generator[Payment, None, None]:
        kwa = {field.name: getattr(self, field.name) for field in dataclasses.fields(self)}

        kwa['calc_date'] = CalcDate(calc_date) if calc_date else None

        yield from PaymentFactory.create('Juros mensais', **kwa)

@dataclasses.dataclass
class Price:
    principal: decimal.Decimal
    apy: decimal.Decimal
    zero_date: datetime.date
    term: int

    insertions: typing.List[Amortization.Bare] = dataclasses.field(default_factory=list)

    def get_payments(self, calc_date: datetime.date | None) -> typing.Generator[Payment, None, None]:
        kwa = {field.name: getattr(self, field.name) for field in dataclasses.fields(self)}

        kwa['calc_date'] = CalcDate(calc_date) if calc_date else None

        yield from PaymentFactory.create('Price', **kwa)

@dataclasses.dataclass
class Livre:
    principal: decimal.Decimal
    apy: decimal.Decimal
    plan_skeleton: str

    vir: VariableIndex | None = None
    insertions: typing.List[Amortization.Bare] = dataclasses.field(default_factory=list)

    def get_payments(self, calc_date: datetime.date | None) -> typing.Generator[Payment, None, None]:
        kwa = {field.name: getattr(self, field.name) for field in dataclasses.fields(self)}
        txt = textwrap.dedent(kwa.pop('plan_skeleton')).strip()

        kwa['amortizations'] = []

        for i, line in enumerate(txt.splitlines()):
            t00 = line.split(',', 3)
            t01 = t00[-1].split(',') if len(t00) == 4 else None

            if i == 0:
                ent = Amortization(date=datetime.date.fromisoformat(t00[0]))

                ent.amortization_ratio = decimal.Decimal()
                ent.amortizes_interest = False

            else:
                ent = Amortization(date=datetime.date.fromisoformat(t00[0]))

                ent.amortization_ratio = decimal.Decimal(t00[1])
                ent.amortizes_interest = t00[2] == 'true'

                if t01 and t01[0] == 'IPCA':
                    ent.price_level_adjustment = Amortization.PriceLevelAdjustment('IPCA')
                    ent.price_level_adjustment.base_date = datetime.date.fromisoformat(t01[1])
                    ent.price_level_adjustment.period = int(t01[2])
                    ent.price_level_adjustment.shift = typing.cast(_PL_SHIFT, t01[3])
                    ent.price_level_adjustment.amortizes_adjustment = t01[4] == 'true'

            kwa['amortizations'].append(ent)

        kwa['calc_date'] = CalcDate(calc_date) if calc_date else None

        yield from PaymentFactory.create('Livre', **kwa)

OPS: t.Dict[str, typing.Union[Bullet, BulletLegacy, JurosMensais, Price, Livre]] = {
    'Villa VIC Pisa - Pré-Fixada - Bullet': BulletLegacy(
        principal=decimal.Decimal('1609500'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 3, 9),
        term=12
    ),

    'Villa VIC Pisa - Pré-Fixada - Juros mensais': JurosMensais(
        principal=decimal.Decimal('890500'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 3, 9),
        term=12
    ),

    'Villa VIC Pisa - CDI - Bullet': Bullet(
        principal=decimal.Decimal('1944500'),
        apy=decimal.Decimal('6.0'),
        zero_date=datetime.date(2022, 3, 9),
        term=12,
        vir=VariableIndex('CDI')
    ),

    'Villa VIC Pisa - CDI - Juros mensais': JurosMensais(
        principal=decimal.Decimal('555500'),
        apy=decimal.Decimal('6.0'),
        zero_date=datetime.date(2022, 3, 9),
        term=12,
        vir=VariableIndex('CDI')
    ),

    'Credlar Chopin - Pré-Fixada - Price': Price(
        principal=decimal.Decimal('481000'),
        apy=decimal.Decimal('19.0'),
        zero_date=datetime.date(2022, 3, 4),
        term=24
    ),

    'Residencial Arnaldo Patrus 2 - Pré-Fixada - Price': Price(
        principal=decimal.Decimal('151000'),
        apy=decimal.Decimal('18.0'),
        zero_date=datetime.date(2022, 3, 4),
        term=24
    ),

    'Felicidade Residencial Clube - Pré-Fixada - Bullet': BulletLegacy(
        principal=decimal.Decimal('978000'),
        apy=decimal.Decimal('13.5'),
        zero_date=datetime.date(2021, 7, 23),
        term=9
    ),

    'Felicidade Residencial Clube - Pré-Fixada - Juros mensais': JurosMensais(
        principal=decimal.Decimal('222000'),
        apy=decimal.Decimal('13.5'),
        zero_date=datetime.date(2021, 7, 23),
        term=9
    ),

    'YUCA SPE V - IPCA - Juros mensais': JurosMensais(
        principal=decimal.Decimal('1844500'),
        apy=decimal.Decimal('7.0'),
        zero_date=datetime.date(2022, 2, 18),
        vir=VariableIndex('IPCA'),
        term=30
    ),

    # Skeleton:
    #
    #  • principal amortization;
    #  • interest amortization;
    #  • price leveling;
    #  • price leveling base date;
    #  • price leveling period;
    #  • price leveling shift mode (M-1, M-2, AUTO);
    #  • price level amortization.
    #
    'ASAD Energia - IPCA - Livre': Livre(
        principal=decimal.Decimal('145000'),
        apy=decimal.Decimal('10.0'),
        vir=VariableIndex('IPCA'),
        plan_skeleton='''
            2022-04-05,0,false
            2022-05-05,0.0130614411,true
            2022-06-05,0.0131655949,true
            2022-07-05,0.0132705792,true
            2022-08-05,0.0133764006,true
            2022-09-05,0.0134830659,true,IPCA,2021-07-01,12,AUTO,true
            2022-10-05,0.0135905818,true,IPCA,2021-07-01,12,AUTO,true
            2022-11-05,0.0136989550,true,IPCA,2021-07-01,12,AUTO,true
            2022-12-05,0.0138081924,true,IPCA,2021-07-01,12,AUTO,true
            2023-01-05,0.0139183008,true,IPCA,2021-07-01,12,AUTO,true
            2023-02-05,0.0140292873,true,IPCA,2021-07-01,12,AUTO,true
            2023-03-05,0.0141411588,true,IPCA,2021-07-01,12,AUTO,true
            2023-04-05,0.0142539224,true,IPCA,2021-07-01,12,AUTO,true
            2023-05-05,0.0143675852,true,IPCA,2021-07-01,12,AUTO,true
            2023-06-05,0.0144821543,true,IPCA,2021-07-01,12,AUTO,true
            2023-07-05,0.0145976371,true,IPCA,2021-07-01,12,AUTO,true
            2023-08-05,0.0147140407,true,IPCA,2021-07-01,12,AUTO,true
            2023-09-05,0.0148313725,true,IPCA,2021-07-01,24,AUTO,true
            2023-10-05,0.0149496400,true,IPCA,2021-07-01,24,AUTO,true
            2023-11-05,0.0150688505,true,IPCA,2021-07-01,24,AUTO,true
            2023-12-05,0.0151890116,true,IPCA,2021-07-01,24,AUTO,true
            2024-01-05,0.0153101309,true,IPCA,2021-07-01,24,AUTO,true
            2024-02-05,0.0154322161,true,IPCA,2021-07-01,24,AUTO,true
            2024-03-05,0.0155552747,true,IPCA,2021-07-01,24,AUTO,true
            2024-04-05,0.0156793147,true,IPCA,2021-07-01,24,AUTO,true
            2024-05-05,0.0158043437,true,IPCA,2021-07-01,24,AUTO,true
            2024-06-05,0.0159303698,true,IPCA,2021-07-01,24,AUTO,true
            2024-07-05,0.0160574008,true,IPCA,2021-07-01,24,AUTO,true
            2024-08-05,0.0161854448,true,IPCA,2021-07-01,24,AUTO,true
            2024-09-05,0.0163145098,true,IPCA,2021-07-01,36,AUTO,true
            2024-10-05,0.0164446040,true,IPCA,2021-07-01,36,AUTO,true
            2024-11-05,0.0165757355,true,IPCA,2021-07-01,36,AUTO,true
            2024-12-05,0.0167079128,true,IPCA,2021-07-01,36,AUTO,true
            2025-01-05,0.0168411440,true,IPCA,2021-07-01,36,AUTO,true
            2025-02-05,0.0169754377,true,IPCA,2021-07-01,36,AUTO,true
            2025-03-05,0.0171108022,true,IPCA,2021-07-01,36,AUTO,true
            2025-04-05,0.0172472461,true,IPCA,2021-07-01,36,AUTO,true
            2025-05-05,0.0173847781,true,IPCA,2021-07-01,36,AUTO,true
            2025-06-05,0.0175234068,true,IPCA,2021-07-01,36,AUTO,true
            2025-07-05,0.0176631409,true,IPCA,2021-07-01,36,AUTO,true
            2025-08-05,0.0178039892,true,IPCA,2021-07-01,36,AUTO,true
            2025-09-05,0.0179459608,true,IPCA,2021-07-01,48,AUTO,true
            2025-10-05,0.0180890644,true,IPCA,2021-07-01,48,AUTO,true
            2025-11-05,0.0182333091,true,IPCA,2021-07-01,48,AUTO,true
            2025-12-05,0.0183787041,true,IPCA,2021-07-01,48,AUTO,true
            2026-01-05,0.0185252584,true,IPCA,2021-07-01,48,AUTO,true
            2026-02-05,0.0186729814,true,IPCA,2021-07-01,48,AUTO,true
            2026-03-05,0.0188218824,true,IPCA,2021-07-01,48,AUTO,true
            2026-04-05,0.0189719708,true,IPCA,2021-07-01,48,AUTO,true
            2026-05-05,0.0191232559,true,IPCA,2021-07-01,48,AUTO,true
            2026-06-05,0.0192757474,true,IPCA,2021-07-01,48,AUTO,true
            2026-07-05,0.0194294550,true,IPCA,2021-07-01,48,AUTO,true
            2026-08-05,0.0195843882,true,IPCA,2021-07-01,48,AUTO,true
            2026-09-05,0.0197405568,true,IPCA,2021-07-01,60,AUTO,true
            2026-10-05,0.0198979708,true,IPCA,2021-07-01,60,AUTO,true
            2026-11-05,0.0200566400,true,IPCA,2021-07-01,60,AUTO,true
            2026-12-05,0.0202165745,true,IPCA,2021-07-01,60,AUTO,true
            2027-01-05,0.0203777843,true,IPCA,2021-07-01,60,AUTO,true
            2027-02-05,0.0205402796,true,IPCA,2021-07-01,60,AUTO,true
            2027-03-05,0.0207040707,true,IPCA,2021-07-01,60,AUTO,true
            2027-04-05,0.0208691707,true,IPCA,2021-07-01,60,AUTO,true
        '''
    ),

    'Ágata - CDI - Livre': Livre(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('6.33'),
        vir=VariableIndex('CDI'),
        plan_skeleton='''
            2021-12-28,0,false
            2022-01-28,0.05555555555,true
            2022-02-28,0.05555555555,true
            2022-03-28,0.05555555555,true
            2022-04-28,0.05555555555,true
            2022-05-28,0.05555555555,true
            2022-06-28,0.05555555555,true
            2022-07-28,0.05555555555,true
            2022-08-28,0.05555555555,true
            2022-09-28,0.05555555555,true
            2022-10-28,0.05555555555,true
            2022-11-28,0.05555555555,true
            2022-12-28,0.05555555555,true
            2023-01-28,0.05555555555,true
            2023-02-28,0.05555555555,true
            2023-03-28,0.05555555555,true
            2023-04-28,0.05555555555,true
            2023-05-28,0.05555555555,true
            2023-06-28,0.05555555555,true
        '''
    ),

    'Bossanova CCB5 - Poupança - Bullet': Bullet(
        principal=decimal.Decimal('1235000'),
        apy=decimal.Decimal('0'),
        zero_date=datetime.date(2022, 6, 23),
        term=120,
        vir=VariableIndex('Poupança', 118)
    ),

    'Bossanova CCB7 - IPCA - Bullet': Bullet(
        principal=decimal.Decimal('176000'),
        apy=decimal.Decimal('0'),
        zero_date=datetime.date(2022, 10, 24),
        term=120,
        vir=VariableIndex('IPCA')
    ),

    'Unique Tower PCI - IPCA - Livre': Livre(  # PCI = Proteção Contra Inflação.
        principal=decimal.Decimal('1100500'),
        apy=decimal.Decimal('11'),
        vir=VariableIndex('IPCA'),
        plan_skeleton='''
            2022-07-08,0,false
            2022-08-08,0.033333333333333335,true,IPCA,2022-07-01,01,M-1,true
            2022-09-08,0.033333333333333335,true,IPCA,2022-07-01,02,M-1,true
            2022-10-08,0.033333333333333335,true,IPCA,2022-07-01,03,M-1,true
            2022-11-08,0.033333333333333335,true,IPCA,2022-07-01,04,M-1,true
            2022-12-08,0.033333333333333335,true,IPCA,2022-07-01,05,M-1,true
            2023-01-08,0.033333333333333335,true,IPCA,2022-07-01,06,M-1,true
            2023-02-08,0.033333333333333335,true,IPCA,2022-07-01,07,M-1,true
            2023-03-08,0.033333333333333335,true,IPCA,2022-07-01,08,M-1,true
            2023-04-08,0.033333333333333335,true,IPCA,2022-07-01,09,M-1,true
            2023-05-08,0.033333333333333335,true,IPCA,2022-07-01,10,M-1,true
            2023-06-08,0.033333333333333335,true,IPCA,2022-07-01,11,M-1,true
            2023-08-08,0.033333333333333335,true,IPCA,2022-07-01,12,M-1,true
            2023-09-08,0.033333333333333335,true,IPCA,2022-07-01,13,M-1,true
            2023-10-08,0.033333333333333335,true,IPCA,2022-07-01,14,M-1,true
            2023-11-08,0.033333333333333335,true,IPCA,2022-07-01,15,M-1,true
            2023-12-08,0.033333333333333335,true,IPCA,2022-07-01,16,M-1,true
            2024-01-08,0.033333333333333335,true,IPCA,2022-07-01,17,M-1,true
            2024-02-08,0.033333333333333335,true,IPCA,2022-07-01,18,M-1,true
            2024-03-08,0.033333333333333335,true,IPCA,2022-07-01,19,M-1,true
            2024-04-08,0.033333333333333335,true,IPCA,2022-07-01,20,M-1,true
            2024-05-08,0.033333333333333335,true,IPCA,2022-07-01,21,M-1,true
            2024-06-08,0.033333333333333335,true,IPCA,2022-07-01,22,M-1,true
            2024-07-08,0.033333333333333335,true,IPCA,2022-07-01,23,M-1,true
            2024-08-08,0.033333333333333335,true,IPCA,2022-07-01,24,M-1,true
            2024-09-08,0.033333333333333335,true,IPCA,2022-07-01,25,M-1,true
            2024-10-08,0.033333333333333335,true,IPCA,2022-07-01,26,M-1,true
            2024-11-08,0.033333333333333335,true,IPCA,2022-07-01,27,M-1,true
            2024-12-08,0.033333333333333335,true,IPCA,2022-07-01,28,M-1,true
            2025-01-08,0.033333333333333335,true,IPCA,2022-07-01,29,M-1,true
            2025-02-08,0.033333333333333335,true,IPCA,2022-07-01,30,M-1,true
        '''
    ),

    'VIC Park Montana e Villa VIC Salerno & Parcelas Amortizadas - CDI - Livre': Livre(
        principal=decimal.Decimal('2000000'),
        apy=decimal.Decimal('6'),
        vir=VariableIndex('CDI'),
        plan_skeleton='''
            2022-07-15,0,false
            2022-08-15,0.02777777777778,true
            2022-09-15,0.02777777777778,true
            2022-10-15,0.02777777777778,true
            2022-11-15,0.02777777777778,true
            2022-12-15,0.02777777777778,true
            2023-01-15,0.02777777777778,true
            2023-02-15,0.02777777777778,true
            2023-03-15,0.02777777777778,true
            2023-04-15,0.02777777777778,true
            2023-05-15,0.02777777777778,true
            2023-06-15,0.02777777777778,true
            2023-07-15,0.02777777777778,true
            2023-08-15,0.02777777777778,true
            2023-09-15,0.02777777777778,true
            2023-10-15,0.02777777777778,true
            2023-11-15,0.02777777777778,true
            2023-12-15,0.02777777777778,true
            2024-01-15,0.02777777777778,true
            2024-02-15,0.02777777777778,true
            2024-03-15,0.02777777777778,true
            2024-04-15,0.02777777777778,true
            2024-05-15,0.02777777777778,true
            2024-06-15,0.02777777777778,true
            2024-07-15,0.02777777777778,true
            2024-08-15,0.02777777777778,true
            2024-09-15,0.02777777777778,true
            2024-10-15,0.02777777777778,true
            2024-11-15,0.02777777777778,true
            2024-12-15,0.02777777777778,true
            2025-01-15,0.02777777777778,true
            2025-02-15,0.02777777777778,true
            2025-03-15,0.02777777777778,true
            2025-04-15,0.02777777777778,true
            2025-05-15,0.02777777777778,true
            2025-06-15,0.02777777777778,true
            2025-07-15,0.02777777777778,true
        '''
    ),

    'Crédito p/ Estudantes de Medicina (Mesada Médicos) - Pré-Fixada - Livre': Livre(
        principal=decimal.Decimal('2000000'),
        apy=decimal.Decimal('21.5'),
        plan_skeleton='''
            2022-08-08,0,false
            2022-09-08,0,false
            2022-10-08,0,false
            2022-11-08,0,false
            2022-12-08,0,false
            2023-01-08,0,false
            2023-02-08,0,false
            2023-03-08,0.02608593235,true
            2023-04-08,0.02651272620,true
            2023-05-08,0.02694650285,true
            2023-06-08,0.02738737656,true
            2023-07-08,0.02783546343,true
            2023-08-08,0.02829088149,true
            2023-09-08,0.02875375067,true
            2023-10-08,0.02922419289,true
            2023-11-08,0.02970233205,true
            2023-12-08,0.03018829408,true
            2024-01-08,0.03068220697,true
            2024-02-08,0.03118420081,true
            2024-03-08,0.03169440780,true
            2024-04-08,0.03221296233,true
            2024-05-08,0.03274000096,true
            2024-06-08,0.03327566252,true
            2024-07-08,0.03382008807,true
            2024-08-08,0.03437342101,true
            2024-09-08,0.03493580706,true
            2024-10-08,0.03550739436,true
            2024-11-08,0.03608833344,true
            2024-12-08,0.03667877731,true
            2025-01-08,0.03727888147,true
            2025-02-08,0.03788880398,true
            2025-03-08,0.03850870548,true
            2025-04-08,0.03913874923,true
            2025-05-08,0.03977910117,true
            2025-06-08,0.04042992996,true
            2025-07-08,0.04109140700,true
            2025-08-08,0.04176370650,true
        '''
    ),

    'Villa das Paineiras - CDI - Livre': Livre(
        principal=decimal.Decimal('3000000'),
        apy=decimal.Decimal('6.5'),
        vir=VariableIndex('CDI'),
        plan_skeleton='''
            2022-09-19,0,false
            2022-10-19,0,false
            2022-11-19,0,false
            2022-12-19,0,false
            2023-01-19,0,false
            2023-02-19,0,false
            2023-03-19,0,false
            2023-04-19,0.05555555555,true
            2023-05-19,0.05555555555,true
            2023-06-19,0.05555555555,true
            2023-07-19,0.05555555555,true
            2023-08-19,0.05555555555,true
            2023-09-19,0.05555555555,true
            2023-10-19,0.05555555555,true
            2023-11-19,0.05555555555,true
            2023-12-19,0.05555555555,true
            2024-01-19,0.05555555555,true
            2024-02-19,0.05555555555,true
            2024-03-19,0.05555555555,true
            2024-04-19,0.05555555555,true
            2024-05-19,0.05555555555,true
            2024-06-19,0.05555555555,true
            2024-07-19,0.05555555555,true
            2024-08-19,0.05555555555,true
            2024-09-19,0.05555555555,true
        '''
    ),

    'Residencial da Mata (15%) - Pré-Fixada - Bullet': BulletLegacy(
        principal=decimal.Decimal('110500.00'),
        apy=decimal.Decimal('15.0'),
        zero_date=datetime.date(2021, 6, 4),
        term=18,
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 10, 6), amortization_ratio=decimal.Decimal('1.0'))
        ]
    ),

    'Residencial da Mata (15%) - Pré-Fixada - Price': Price(
        principal=decimal.Decimal('115000.00'),
        apy=decimal.Decimal('15.0'),
        zero_date=datetime.date(2021, 6, 4),
        term=18,
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 10, 6), amortization_ratio=decimal.Decimal('0.1217411843853222616888311797'))
        ]
    ),

    'Residencial da Mata (15.5%) - Pré-Fixada - Bullet': BulletLegacy(
        principal=decimal.Decimal('634000.00'),
        apy=decimal.Decimal('15.5'),
        zero_date=datetime.date(2021, 6, 4),
        term=18,
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 10, 6), amortization_ratio=decimal.Decimal('1.0'))
        ]
    ),

    'Residencial da Mata (15.5%) - Pré-Fixada - Price': Price(
        principal=decimal.Decimal('633500.00'),
        apy=decimal.Decimal('15.5'),
        zero_date=datetime.date(2021, 6, 4),
        term=18,
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 10, 6), amortization_ratio=decimal.Decimal('0.1220798914196176313899832699'))
        ]
    )
}

XOPS = {
    'Teste 01 (aniversário) - Pré-Fixada - Juros mensais': JurosMensais(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('20.0'),
        zero_date=datetime.date(2020, 12, 1),
        anniversary_date=datetime.date(2020, 12, 15),
        term=12
    ),

    'Teste 02 (aniversário) - CDI - Livre': Livre(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('15'),
        vir=VariableIndex('CDI'),
        plan_skeleton='''
            2021-06-06,0,false
            2021-07-01,0.05555555555,true
            2021-08-01,0.05555555555,true
            2021-09-01,0.05555555555,true
            2021-10-01,0.05555555555,true
            2021-11-01,0.05555555555,true
            2021-12-01,0.05555555555,true
            2022-01-01,0.05555555555,true
            2022-02-01,0.05555555555,true
            2022-03-01,0.05555555555,true
            2022-04-01,0.05555555555,true
            2022-05-01,0.05555555555,true
            2022-06-01,0.05555555555,true
            2022-07-01,0.05555555555,true
            2022-08-01,0.05555555555,true
            2022-09-01,0.05555555555,true
            2022-10-01,0.05555555555,true
            2022-11-01,0.05555555555,true
            2022-12-01,0.05555555555,true
        '''
    ),

    'Teste 03 (adiantamento PARCIAL) - Pré-Fixada - Bullet': Bullet(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 5, 21), amortization_ratio=decimal.Decimal('0.50')),
            Amortization.Bare(date=datetime.date(2022, 8, 12), amortization_ratio=decimal.Decimal('0.25'))
        ]
    ),

    'Teste 04 (adiantamento PARCIAL) - CDI - Bullet': Bullet(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('10.0'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        vir=VariableIndex('CDI'),
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 5, 21), amortization_ratio=decimal.Decimal('0.50')),
            Amortization.Bare(date=datetime.date(2022, 8, 12), amortization_ratio=decimal.Decimal('0.25'))
        ]
    ),

    'Teste 05 (adiantamento PARCIAL) - Pré-Fixada - Juros mensais': JurosMensais(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 5, 21), amortization_ratio=decimal.Decimal('0.50')),
            Amortization.Bare(date=datetime.date(2022, 8, 12), amortization_ratio=decimal.Decimal('0.25'))
        ]
    ),

    'Teste 06 (adiantamento PARCIAL) - CDI - Juros mensais': JurosMensais(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('10.0'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        vir=VariableIndex('CDI'),
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 5, 21), amortization_ratio=decimal.Decimal('0.50')),
            Amortization.Bare(date=datetime.date(2022, 8, 12), amortization_ratio=decimal.Decimal('0.25'))
        ]
    ),

    'Teste 07 (adiantamento PARCIAL) - IPCA - Juros mensais': JurosMensais(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('10.0'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        vir=VariableIndex('IPCA'),
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 5, 21), amortization_ratio=decimal.Decimal('0.50')),
            Amortization.Bare(date=datetime.date(2022, 8, 12), amortization_ratio=decimal.Decimal('0.25'))
        ]
    ),

    'Teste 08 (adiantamento PARCIAL) - Pré-Fixada - Price': Price(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 5, 21), amortization_ratio=decimal.Decimal('0.10')),
            Amortization.Bare(date=datetime.date(2022, 8, 12), amortization_ratio=decimal.Decimal('0.10'))
        ]
    ),

    'Teste 09 (adiantamento PARCIAL) - Pré-Fixada - Livre': Livre(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('15.5'),
        plan_skeleton='''
            2022-02-10,0,false
            2022-03-10,0.05555555555,true
            2022-04-10,0.05555555555,true
            2022-05-10,0.05555555555,true
            2022-06-10,0.05555555555,true
            2022-07-10,0.05555555555,true
            2022-08-10,0.05555555555,true
            2022-09-10,0.05555555555,true
            2022-10-10,0.05555555555,true
            2022-11-10,0.05555555555,true
            2022-12-10,0.05555555555,true
            2023-01-10,0.05555555555,true
            2023-02-10,0.05555555555,true
            2023-03-10,0.05555555555,true
            2023-04-10,0.05555555555,true
            2023-05-10,0.05555555555,true
            2023-06-10,0.05555555555,true
            2023-07-10,0.05555555555,true
            2023-08-10,0.05555555555,true
        ''',
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 5, 21), amortization_ratio=decimal.Decimal('0.10')),
            Amortization.Bare(date=datetime.date(2022, 8, 12), amortization_ratio=decimal.Decimal('0.10')),
            Amortization.Bare(date=datetime.date(2022, 11, 15), amortization_ratio=decimal.Decimal('0.10')),
            Amortization.Bare(date=datetime.date(2022, 11, 25), amortization_ratio=decimal.Decimal('0.10'))
        ]
    ),

    'Teste 10 (adiantamento PARCIAL + aniversário) - Pré-Fixada - Livre': Livre(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('15.5'),
        plan_skeleton='''
            2022-02-10,0,false
            2022-03-05,0.08333333333,true
            2022-04-05,0.08333333333,true
            2022-05-05,0.08333333333,true
            2022-06-05,0.08333333333,true
            2022-07-05,0.08333333333,true
            2022-08-05,0.08333333333,true
            2022-09-05,0.08333333333,true
            2022-10-05,0.08333333333,true
            2022-11-05,0.08333333333,true
            2022-12-05,0.08333333333,true
            2023-01-05,0.08333333333,true
            2023-02-05,0.08333333333,true
        ''',
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 2, 21), amortization_ratio=decimal.Decimal('0.10')),
            Amortization.Bare(date=datetime.date(2022, 8, 12), amortization_ratio=decimal.Decimal('0.10')),
            Amortization.Bare(date=datetime.date(2022, 11, 15), amortization_ratio=decimal.Decimal('0.10'))
        ]
    ),

    'Teste 11 (adiantamento TOTAL) - Pré-Fixada - Bullet': Bullet(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 6, 30), amortization_ratio=decimal.Decimal('1.00'))
        ]
    ),

    'Teste 12 (adiantamento TOTAL) - Pré-Fixada - Juros mensais': JurosMensais(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 6, 30), amortization_ratio=decimal.Decimal('1.00'))
        ]
    ),

    'Teste 13 (adiantamento TOTAL) - Pré-Fixada - Price': Price(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            # Se caracteriaza pela antecipação total da operação por amortizar o percentual restante do principal na data especificada.
            Amortization.Bare(date=datetime.date(2022, 6, 30), amortization_ratio=decimal.Decimal('0.6853392053203554776089855225'))
        ]
    ),

    'Teste 14 (adiantamento TOTAL) - Pré-Fixada - Livre': Livre(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        plan_skeleton='''
            2022-02-10,0,false
            2022-03-10,0.08333333333,true
            2022-04-10,0.08333333333,true
            2022-05-10,0.08333333333,true
            2022-06-10,0.08333333333,true
            2022-07-10,0.08333333333,true
            2022-08-10,0.08333333333,true
            2022-09-10,0.08333333333,true
            2022-10-10,0.08333333333,true
            2022-11-10,0.08333333333,true
            2022-12-10,0.08333333333,true
            2023-01-10,0.08333333333,true
            2023-02-10,0.08333333333,true
        ''',
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 6, 30), amortization_ratio=decimal.Decimal('0.66666666668'))
        ]
    ),

    'Teste 15 (adiantamento PARCIAL + TOTAL) - Pré-Fixada - Bullet': Bullet(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('21'),
        zero_date=datetime.date(2022, 8, 1),
        term=12,
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 12, 1), amortization_ratio=decimal.Decimal('0.20')),
            Amortization.Bare(date=datetime.date(2023, 2, 1), amortization_ratio=decimal.Decimal('0.30')),
            Amortization.Bare(date=datetime.date(2023, 6, 1), amortization_ratio=decimal.Decimal('0.50'))
        ]
    ),

    'Teste 16 (adiantamento PARCIAL + TOTAL) - IPCA - Bullet': Bullet(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('11'),
        zero_date=datetime.date(2021, 1, 1),
        term=12,
        vir=VariableIndex('IPCA'),
        insertions=[
            Amortization.Bare(date=datetime.date(2021, 4, 1), amortization_ratio=decimal.Decimal('0.20')),
            Amortization.Bare(date=datetime.date(2021, 6, 1), amortization_ratio=decimal.Decimal('0.30')),
            Amortization.Bare(date=datetime.date(2021, 8, 1), amortization_ratio=decimal.Decimal('0.50'))
        ]
    ),

    'Teste 17 (adiantamento PARCIAL + TOTAL) - IPCA - Livre': Livre(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('10'),
        plan_skeleton='''
            2022-09-16,0,false
            2022-10-16,0.08333333333,true,IPCA,2021-09-01,01,AUTO,true
            2022-11-16,0.08333333333,true,IPCA,2021-09-01,02,AUTO,true
            2022-12-16,0.08333333334,true,IPCA,2021-09-01,03,AUTO,true
            2022-01-16,0.08333333333,true,IPCA,2021-09-01,04,AUTO,true
            2022-02-16,0.08333333333,true,IPCA,2021-09-01,05,AUTO,true
            2022-03-16,0.08333333334,true,IPCA,2021-09-01,06,AUTO,true
            2022-04-16,0.08333333333,true,IPCA,2021-09-01,06,AUTO,true
            2022-05-16,0.08333333333,true,IPCA,2021-09-01,08,AUTO,true
            2022-06-16,0.08333333334,true,IPCA,2021-09-01,09,AUTO,true
            2022-07-16,0.08333333333,true,IPCA,2021-09-01,10,AUTO,true
            2022-08-16,0.08333333333,true,IPCA,2021-09-01,11,AUTO,true
            2022-09-16,0.08333333334,true,IPCA,2021-09-01,12,AUTO,true
        ''',
        insertions=[
            Amortization.Bare(date=datetime.date(2022, 11, 1), amortization_ratio=decimal.Decimal('0.10')),
            Amortization.Bare(date=datetime.date(2022, 12, 1), amortization_ratio=decimal.Decimal('0.10')),
            Amortization.Bare(date=datetime.date(2023, 1, 1), amortization_ratio=decimal.Decimal('0.5781818181828357024793320011'))
        ]
    ),

    'Teste 18 (adiantamento PARCIAL + carência) - CDI - Livre': Livre(
        principal=decimal.Decimal('3000000'),
        apy=decimal.Decimal('6.5'),
        vir=VariableIndex('CDI'),
        plan_skeleton='''
            2022-09-15,0,false
            2022-10-15,0,false
            2022-11-15,0,false
            2022-12-15,0,false
            2023-01-15,0,false
            2023-02-15,0,false
            2023-03-15,0,false
            2023-04-15,0.05555555555,true
            2023-05-15,0.05555555555,true
            2023-06-15,0.05555555555,true
            2023-07-15,0.05555555555,true
            2023-08-15,0.05555555555,true
            2023-09-15,0.05555555555,true
            2023-10-15,0.05555555555,true
            2023-11-15,0.05555555555,true
            2023-12-15,0.05555555555,true
            2024-01-15,0.05555555555,true
            2024-02-15,0.05555555555,true
            2024-03-15,0.05555555555,true
            2024-04-15,0.05555555555,true
            2024-05-15,0.05555555555,true
            2024-06-15,0.05555555555,true
            2024-07-15,0.05555555555,true
            2024-08-15,0.05555555555,true
            2024-09-15,0.05555555555,true
        ''',
        insertions=[
            Amortization.Bare(date=datetime.date(2023, 5, 5), amortization_ratio=decimal.Decimal('0.40'))
        ]
    )
}

In [ ]:
# Python.
import random
import decimal
import datetime
import collections

# Libs.
import pandas
import typing
import ipywidgets

# Lista padrão de operações. Pode assumir os valores "OPS" e "XOPS".
_DEFAULT_OP_LST = OPS

# Valor padrão no componente de operações.
_DEFAULT_OP = random.choice(list(_DEFAULT_OP_LST.keys()))

# Valor padrão para o principal.
_DEFAULT_PRINCIPAL = 10000

def update_apy(event: typing.Any) -> None:
    if event['type'] == 'change' and event['name'] == 'value':
        op = _DEFAULT_OP_LST[form['op'].value]

        if 'CDI' in form['op'].value:
            grid[0, 1].value = f'APY: {op.apy}% + CDI; TERM: {getattr(op, "term", "???")}'

        else:
            grid[0, 1].value = f'APY: {op.apy}%; TERM: {getattr(op, "term", "???")}'

def show_schedule(
    op: str,
    principal: int,
    calc_date: datetime.datetime | None
) -> pandas.DataFrame:
    buf: t.Dict[str, typing.Any] = collections.defaultdict(list)

    for x in _DEFAULT_OP_LST[op].get_payments(calc_date):
        buf['nº'].append(x.no)
        buf['date'].append(x.date)
        buf['raw'].append(x.raw)
        buf['tax'].append(x.tax)
        buf['net'].append(x.net)
        buf['gain'].append(x.gain)
        buf['amort'].append(x.amort)

        if hasattr(x, 'pla'):
            buf['pla'].append(x.pla)

        else:
            buf['pla'].append(decimal.Decimal())

        buf['bal'].append(x.bal)

    df = pandas.DataFrame(buf).set_index('nº')

    df['date'] = pandas.to_datetime(df['date'])

    display(df)  # type: ignore # noqa

form: t.Dict[str, t.Any] = {'op': {}}
grid = ipywidgets.GridspecLayout(2, 2)
opt0 = _DEFAULT_OP_LST[_DEFAULT_OP]

# 1.1. Operation selector.
form['op']['options'] = [k for k in _DEFAULT_OP_LST]
form['op']['description'] = 'Operação'
form['op']['value'] = _DEFAULT_OP
form['op'] = ipywidgets.Dropdown(**form['op'])
form['op'].observe(update_apy, names='value')

# 1.2. Operation information.
form['apy'] = ipywidgets.Label(f'APY: {opt0.apy}% TERM: {getattr(opt0, "term", "???")}')

# 1.3. Principal.
form['principal'] = {}
form['principal']['min'] = 100
form['principal']['max'] = 3000000
form['principal']['step'] = 500
form['principal']['value'] = _DEFAULT_PRINCIPAL
form['principal']['description'] = 'Principal'
form['principal']['continuous_update'] = False
form['principal'] = ipywidgets.IntSlider(**form['principal'])

# 1.4. Calculation date.
form['calc_date'] = ipywidgets.DatePicker(description='Calc Date')

# 2. Create the widgets Grid.
grid[0, 0] = form['op']
grid[0, 1] = form.pop('apy')
grid[1, 0] = form['calc_date']
grid[1, 1] = form['principal']

# 3. Draw the toolbar (widgets).
ipywidgets.VBox([grid, ipywidgets.interactive_output(show_schedule, form)])

# Notas

Para garantir a qualidade do código instale as seguintes bibliotecas.

    python -m pip install nbqa flake8 mypy

Para validar padrão de codificação, rodar o Flake8.

    nbqa flake8 support/lab-000.ipynb

Para validar tipos de dados.

    nbqa mypy support/lab-000.ipynb --ignore-missing-imports --strict

<br/>

---

Rafael Viotti &lt;viotti@inco.vc&gt; e Gustavo Zille &lt;gustavo.zille@inco.vc&gt;<br/>
Março 2022